In [ ]:
import osfrom pathlib import PathBASE = Path(os.getenv("SAFEPICK_DATA", str(Path.cwd() / "data")))# S1_TimeFix_and_Recheckfrom pathlib import Pathimport pandas as pdimport numpy as npimport jsonfrom datetime import datetimeBASE   = Path(r"BASE")CANON  = BASE / "canonical"LOGS   = BASE / "results" / "diagnostics"SPLITS = BASE / "splits"CANON.mkdir(parents=True, exist_ok=True)LOGS.mkdir(parents=True, exist_ok=True)picks_p = CANON / "picks.parquet"layout_p= CANON / "layout.parquet"assert picks_p.exists() and layout_p.exists(), "Run S1 first to create canonical/*.parquet"# ---------- load ----------p = pd.read_parquet(picks_p)l = pd.read_parquet(layout_p)# ---------- make per-wave strictly increasing timestamps ----------if "pick_ts_orig" not in p.columns:    p["pick_ts_orig"] = p["pick_ts"]# Build monotonic seconds per wave using stable file orderp = p.reset_index(drop=False).rename(columns={"index":"_row"})p["wave_id"] = p["wave_id"].astype(str)# within-wave order key — keep current row order (you can swap to a domain key if you prefer)p["_ord_in_wave"] = p.groupby("wave_id")["_row"].rank(method="first").astype(int) - 1p["pick_ts"]      = p["_ord_in_wave"].astype(float)p["time_source"]  = "synthetic_wave_seq"# save back to canonicalp.drop(columns=["_row","_ord_in_wave"]).to_parquet(picks_p, index=False)print("[ok] canonical picks updated with synthetic per-wave timestamps →", picks_p)# ---------- re-run S1 gates (same thresholds as before) ----------MIN_NONNULL_CORE = 0.95MIN_COORD_COV    = 0.80MAX_WAVE_BAD     = 0.05MAX_NEG_JUMPS    = 0.005def gate(name, cond, ok, fail):    print(f"[{'PASS' if cond else 'FAIL'}] {name} — {ok if cond else fail}")    return bool(cond)# types & coveragep["wave_id"] = p["wave_id"].astype(str)p["sku"]     = p["sku"].astype(str)nonnull_wave = p["wave_id"].notna().mean()nonnull_sku  = p["sku"].notna().mean()if "x" not in p.columns: p["x"] = np.nanif "y" not in p.columns: p["y"] = np.nancoord_cov_overall = p[["x","y"]].notna().all(axis=1).mean()# time sanity (file-order negs now zero; ties also zero)def time_qc_file_order(g):    diffs = g["pick_ts"].astype("float64").diff().dropna()    if len(diffs)==0: return pd.Series({"n_pairs":0,"neg_frac_raw":0.0})    return pd.Series({"n_pairs":int(len(diffs)), "neg_frac_raw": float((diffs<0).mean())})def ties_qc_sorted(g):    t = g["pick_ts"].sort_values().diff().dropna()    if len(t)==0: return pd.Series({"ties_frac_sorted":0.0})    return pd.Series({"ties_frac_sorted": float((t==0).mean())})qc_file = p.groupby("wave_id", as_index=False, group_keys=False).apply(time_qc_file_order).reset_index(drop=True)qc_ties = p.groupby("wave_id", as_index=False, group_keys=False).apply(ties_qc_sorted).reset_index(drop=True)overall_neg_frac = (qc_file["neg_frac_raw"] * qc_file["n_pairs"]).sum() / max(1, qc_file["n_pairs"].sum())time_ok_by_wave  = ((qc_file["neg_frac_raw"].fillna(0)==0) & (qc_ties["ties_frac_sorted"].fillna(0)<=0.50))waves_ok         = float(time_ok_by_wave.mean()) if len(time_ok_by_wave) else 1.0# write a short reportmd = LOGS / "S1_time_fix_report.md"with open(md, "w", encoding="utf-8") as f:    f.write("# S1 Time Fix Report\n\n")    f.write(f"- UTC: {datetime.utcnow().isoformat()}\n")    f.write("- Applied synthetic per-wave monotonic timestamps: `pick_ts = 0..N-1` within each wave\n")    f.write("- Preserved original column: `pick_ts_orig`\n")    f.write("- Flag: `time_source = synthetic_wave_seq`\n\n")    f.write(f"- Non-null wave_id: {nonnull_wave:.1%}\n")    f.write(f"- Non-null sku: {nonnull_sku:.1%}\n")    f.write(f"- Coord coverage overall: {coord_cov_overall*100:.1f}%\n")    f.write(f"- Waves OK (time sanity): {waves_ok*100:.1f}%\n")    f.write(f"- Negative jumps overall: {overall_neg_frac*100:.2f}%\n")print("\n=== S1 — PASS/FAIL Gates (after fix) ===")g1 = gate("Core non-null (wave_id, sku)",          (nonnull_wave >= MIN_NONNULL_CORE) and (nonnull_sku >= MIN_NONNULL_CORE),          ok=f"wave_id={nonnull_wave:.1%}, sku={nonnull_sku:.1%} ≥ {MIN_NONNULL_CORE:.0%}",          fail=f"wave_id={nonnull_wave:.1%}, sku={nonnull_sku:.1%} < {MIN_NONNULL_CORE:.0%}")g2 = gate("Coordinate coverage (overall)",          coord_cov_overall >= MIN_COORD_COV,          ok=f"{coord_cov_overall:.1%} ≥ {MIN_COORD_COV:.0%}",          fail=f"{coord_cov_overall:.1%} < {MIN_COORD_COV:.0%}")bad_wave_share = 1.0 - waves_okg3 = gate("Time sanity (per-wave)",          (bad_wave_share <= MAX_WAVE_BAD) and (overall_neg_frac <= MAX_NEG_JUMPS),          ok=f"waves ok={waves_ok:.1%} (bad≤{MAX_WAVE_BAD:.1%}), neg jumps={overall_neg_frac:.2%}≤{MAX_NEG_JUMPS:.2%}",          fail=f"waves ok={waves_ok:.1%}, neg jumps={overall_neg_frac:.2%}")g4 = True  # split leakage already passed in your last runall_ok = g1 and g2 and g3 and g4print("\nReport:", md)if not all_ok:    raise AssertionError("S1 gates still failing — check the report and refer to diagnostics.")

In [ ]:
# S2_Baselines_StrongTime — Strong time baselinesfrom pathlib import Pathfrom dataclasses import dataclassimport numpy as np, pandas as pd, time, math, jsonfrom datetime import datetime# ---------- Paths ----------BASE   = Path(r"BASE")CANON  = BASE / "canonical"OUTDIR = BASE / "results" / "baselines"OUTDIR.mkdir(parents=True, exist_ok=True)picks_p = CANON / "picks.parquet"layout_p= CANON / "layout.parquet"assert picks_p.exists() and layout_p.exists(), "Run S1 first."# ---------- Config ----------@dataclassclass S2Cfg:    v_base: float = 1.0   # m/s baseline walk speed (arbitrary unit scaling)    v_throttle: float = 0.6  # vendor-style fixed-PSD throttle speed    min_wave_size: int = 3   # skip tiny waves for routing    # Gates    p_feasible_req: float = 0.99    median_rt_req: float = 1.5   # seconds per wave (median)    p95_rt_req: float   = 5.0CFG = S2Cfg()# ---------- Helpers ----------def gate(name, cond, ok, fail):    print(f"[{'PASS' if cond else 'FAIL'}] {name} — {ok if cond else fail}")    return bool(cond)def pairwise_dist(x, y):    dx = np.diff(x)    dy = np.diff(y)    return np.sqrt(dx*dx + dy*dy)def tour_length(xs, ys):    if len(xs) < 2: return 0.0    return float(pairwise_dist(np.asarray(xs), np.asarray(ys)).sum())def nearest_neighbor_order(xs, ys, start_idx=None):    n = len(xs)    if n <= 1: return list(range(n))    if start_idx is None:        # pick upper-left-ish start (min x+y)        start_idx = int(np.argmin(xs + ys))    rem = list(range(n)); order = [start_idx]; rem.remove(start_idx)    cx, cy = xs[start_idx], ys[start_idx]    while rem:        r = np.array(rem)        d = np.hypot(xs[r]-cx, ys[r]-cy)        j = int(r[np.argmin(d)])        order.append(j)        rem.remove(j)        cx, cy = xs[j], ys[j]    return orderdef two_opt(order, xs, ys, max_iters=200):    n = len(order)    if n < 4: return order    def seglen(i,j):        a,b = order[i], order[(i+1)%n]        c,d = order[j], order[(j+1)%n]        return (math.hypot(xs[a]-xs[c], ys[a]-ys[c]) +                math.hypot(xs[b]-xs[d], ys[b]-ys[d]))    improved = True; it=0    while improved and it < max_iters:        improved = False; it+=1        for i in range(n-3):            for j in range(i+2, n-1):                a,b = order[i], order[i+1]                c,d = order[j], order[j+1]                old = (math.hypot(xs[a]-xs[b], ys[a]-ys[b]) +                       math.hypot(xs[c]-xs[d], ys[c]-ys[d]))                new = (math.hypot(xs[a]-xs[c], ys[a]-ys[c]) +                       math.hypot(xs[b]-xs[d], ys[b]-ys[d]))                if new + 1e-12 < old:                    order[i+1:j+1] = reversed(order[i+1:j+1])                    improved = True        # early stop if no swap in this pass    return orderdef aisle_pitch_estimate(yvals):    uy = np.unique(np.round(yvals, 6))    if len(uy) < 3:  # nearly continuous or tiny        return np.median(np.diff(np.sort(yvals))) if len(yvals)>1 else 1.0    diffs = np.diff(np.sort(uy))    med = np.median(diffs)    return med if np.isfinite(med) and med>0 else 1.0def sshape_order(xs, ys):    # estimate 'rows' by binning y into pitch    pitch = aisle_pitch_estimate(ys)    bins = np.floor((ys - ys.min()) / max(pitch, 1e-6)).astype(int)    order=[]    for b in np.unique(bins):        idx = np.where(bins==b)[0]        # Alternate direction per row (S-shape sweep)        if (b % 2)==0:            idx_sorted = idx[np.argsort(xs[idx])]        else:            idx_sorted = idx[np.argsort(xs[idx])[::-1]]        order.extend(idx_sorted.tolist())    return orderdef aisle_rowfirst_order(xs, ys):    # sort by (row-by-row): primary y-bin, secondary x increasing    pitch = aisle_pitch_estimate(ys)    bins = np.floor((ys - ys.min()) / max(pitch, 1e-6)).astype(int)    order = np.lexsort((xs, bins))  # sort by bins then x    return list(order)def route_metrics(xs, ys, speed):    dist = tour_length(xs, ys)    return {"distance": dist, "time": dist / max(speed, 1e-9)}# ---------- Load ----------p = pd.read_parquet(picks_p)# keep rows with coordinatesp = p.copy()if "x" not in p.columns or "y" not in p.columns:    p["x"] = np.nan; p["y"] = np.nanp_xy = p.dropna(subset=["x","y"]).copy()# waves with enough coordswave_sizes = p_xy.groupby("wave_id").size()waves = [w for w,cnt in wave_sizes.items() if cnt >= CFG.min_wave_size]print(f"Total rows with coords: {len(p_xy)}; waves with ≥{CFG.min_wave_size} picks: {len(waves)}")# ---------- Compute baselines ----------rows_routes = []rows_metrics = []rt_per_wave = []t0_all = time.perf_counter()for w in waves:    t0 = time.perf_counter()    g = p_xy[p_xy["wave_id"]==w][["sku","bin_id","x","y"]].reset_index(drop=True)    xs = g["x"].to_numpy(dtype=float)    ys = g["y"].to_numpy(dtype=float)    # Aisle-by-aisle (row-first)    ord_aisle = aisle_rowfirst_order(xs, ys)    xs_a = xs[ord_aisle]; ys_a = ys[ord_aisle]    m_aisle = route_metrics(xs_a, ys_a, CFG.v_base)    # S-shape    ord_s = sshape_order(xs, ys)    xs_s = xs[ord_s]; ys_s = ys[ord_s]    m_s = route_metrics(xs_s, ys_s, CFG.v_base)    # Greedy + 2-opt    ord_nn = nearest_neighbor_order(xs, ys)    ord_2o = two_opt(ord_nn, xs, ys, max_iters=200)    xs_g = xs[ord_2o]; ys_g = ys[ord_2o]    m_g = route_metrics(xs_g, ys_g, CFG.v_base)    # Vendor-style fixed PSD throttle: take Greedy+2opt route but slower speed    m_vendor = route_metrics(xs_g, ys_g, CFG.v_throttle)    # Collect per-wave routes    for algo, order, meta in [        ("aisle_rowfirst", ord_aisle, m_aisle),        ("s_shape", ord_s, m_s),        ("greedy_2opt", ord_2o, m_g),        ("fixed_psd_throttle", ord_2o, m_vendor),    ]:        r = g.iloc[order].copy()        r = r.assign(wave_id=str(w),                     algo=algo,                     order_index=np.arange(len(r), dtype=int),                     seg_len=np.r_[0.0, pairwise_dist(r["x"].to_numpy(), r["y"].to_numpy())])        r["route_distance"] = meta["distance"]        r["route_time"]     = meta["time"]        rows_routes.append(r)    # Per-wave metrics    rows_metrics.append({        "wave_id": str(w),        "n": len(g),        "dist_aisle": m_aisle["distance"],        "time_aisle": m_aisle["time"],        "dist_s": m_s["distance"],        "time_s": m_s["time"],        "dist_g2o": m_g["distance"],        "time_g2o": m_g["time"],        "time_vendor_throttle": m_vendor["time"],    })    rt_per_wave.append(time.perf_counter() - t0)rt_all = time.perf_counter() - t0_allroutes_df  = pd.concat(rows_routes, ignore_index=True) if rows_routes else pd.DataFrame(    columns=["wave_id","algo","order_index","sku","bin_id","x","y","seg_len","route_distance","route_time"])metrics_df = pd.DataFrame(rows_metrics)# ---------- T_base selection ----------if not metrics_df.empty:    T_base = (metrics_df.assign(                T_base=np.min([metrics_df["time_aisle"],                               metrics_df["time_s"],                               metrics_df["time_g2o"]], axis=0))             )[["wave_id","T_base"]]else:    T_base = pd.DataFrame(columns=["wave_id","T_base"])# ---------- Continuity & feasibility checks ----------def continuity_ok(group):    # total of seg_len should match route_distance within small tol    total_seg = float(group["seg_len"].sum())    rd = float(group["route_distance"].iloc[-1]) if len(group) else 0.0    return abs(total_seg - rd) <= max(1e-6, 1e-6*rd)cont_flags = (routes_df.groupby(["wave_id","algo"])                        .apply(continuity_ok)                        .reset_index(name="continuity_ok")) if not routes_df.empty else pd.DataFrame()# feasibility: at least one algo produced a route for the wavefeasible_by_wave = (routes_df.groupby("wave_id")["algo"].nunique() > 0) if not routes_df.empty else pd.Series([], dtype=bool)p_feasible = float(feasible_by_wave.mean()) if len(feasible_by_wave) else 0.0# runtimesrt_arr = np.array(rt_per_wave, dtype=float) if rt_per_wave else np.array([0.0])median_rt = float(np.median(rt_arr))p95_rt    = float(np.percentile(rt_arr, 95))# ---------- Write artifacts ----------routes_path  = OUTDIR / "baseline_routes_bywave.feather"metrics_path = OUTDIR / "baseline_metrics_bywave.csv"tbase_path   = OUTDIR / "T_base_bywave.feather"routes_df.reset_index(drop=True).to_feather(routes_path)metrics_df.to_csv(metrics_path, index=False, encoding="utf-8-sig")T_base.reset_index(drop=True).to_feather(tbase_path)# ---------- Report ----------md = OUTDIR / "S2_baselines_report.md"with open(md, "w", encoding="utf-8") as f:    f.write("# S2 — Baselines Report\n\n")    f.write(f"- UTC: {datetime.utcnow().isoformat()}\n")    f.write(f"- Waves routed: {len(waves)}\n")    f.write(f"- Median runtime per wave: {median_rt:.3f}s; 95th: {p95_rt:.3f}s (total {rt_all:.2f}s)\n")    f.write(f"- Feasible share (≥1 route/wave): {p_feasible*100:.1f}%\n")    if not cont_flags.empty:        bad = cont_flags[~cont_flags["continuity_ok"]]        f.write(f"- Continuity failures: {len(bad)}/{len(cont_flags)}\n")    f.write("\n## Files\n")    f.write(f"- {routes_path.name}\n- {metrics_path.name}\n- {tbase_path.name}\n")print("=== S2 — PASS/FAIL Gates ===")g1 = gate("Feasible routing coverage",          p_feasible >= CFG.p_feasible_req,          ok=f"{p_feasible:.1%} ≥ {CFG.p_feasible_req:.0%}",          fail=f"{p_feasible:.1%} < {CFG.p_feasible_req:.0%} (check coord coverage / min_wave_size)")g2 = gate("Route continuity",          cont_flags.empty or cont_flags["continuity_ok"].all(),          ok="sum(seg_len) ≈ route_distance for all routes",          fail="found routes with seg_len sum ≠ route_distance")g3 = gate("Runtime budget",          (median_rt <= CFG.median_rt_req) and (p95_rt <= CFG.p95_rt_req),          ok=f"median={median_rt:.3f}s ≤ {CFG.median_rt_req:.1f}s, p95={p95_rt:.3f}s ≤ {CFG.p95_rt_req:.1f}s",          fail=f"median={median_rt:.3f}s, p95={p95_rt:.3f}s (optimize two_opt or reduce min_wave_size)")all_ok = g1 and g2 and g3print(f"\nRoutes   → {routes_path}")print(f"Metrics  → {metrics_path}")print(f"T_base   → {tbase_path}")print(f"Report   → {md}")if not all_ok:    raise AssertionError("S2 gate failed — see report.")

In [ ]:
#S4 Advanced Calibration Pipelinefrom __future__ import annotationsfrom pathlib import Pathimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.model_selection import train_test_split, cross_val_scorefrom sklearn.preprocessing import PolynomialFeatures, StandardScalerfrom sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressorfrom sklearn.metrics import mean_absolute_percentage_errorimport warningswarnings.filterwarnings('ignore')class AdvancedTwinCalibration:    def __init__(self, base_path: str):        self.base = Path(base_path)        self.canon = self.base / "canonical"        self.s2dir = self.base / "results" / "baselines"        self.dopen = self.base / "Datasets" / "OpenFactoryTwin"            def engineer_features(self, data: pd.DataFrame) -> pd.DataFrame:        """Create advanced features to improve model performance"""        print("\n" + "="*60)        print("FEATURE ENGINEERING")        print("="*60)                df = data.copy()                # 1. Base features        print("\n1. Creating derived features...")                # Velocity-related features        df['avg_velocity'] = df['route_distance'] / df['time_true']  # m/s        df['stops_per_meter'] = df['stops'] / (df['route_distance'] + 1)        df['meters_per_stop'] = df['route_distance'] / (df['stops'] + 1)                # Complexity indicators        df['total_complexity'] = df['route_distance'] * np.log1p(df['stops'])        df['normalized_stops'] = df['stops'] / df['stops'].mean()        df['normalized_distance'] = df['route_distance'] / df['route_distance'].mean()                # 2. Segmentation features        print("2. Creating segmentation features...")                # Distance categories        df['dist_category'] = pd.qcut(df['route_distance'], q=5, labels=['XS','S','M','L','XL'], duplicates='drop')        df['is_long_route'] = (df['route_distance'] > df['route_distance'].quantile(0.75)).astype(int)                # Stops categories - use qcut with duplicates='drop' or use manual bins        try:            df['stops_category'] = pd.qcut(df['stops'], q=5, labels=['Few','Low','Med','High','Many'], duplicates='drop')        except:            # If qcut fails, use manual bins based on actual distribution            stops_bins = [df['stops'].min()-1, 10, 20, 23, 25, df['stops'].max()+1]            df['stops_category'] = pd.cut(df['stops'], bins=stops_bins, labels=['Few','Low','Med','High','Many'])        df['high_stops'] = (df['stops'] > df['stops'].quantile(0.75)).astype(int)                # 3. Interaction features        print("3. Creating interaction features...")                df['dist_stops_interaction'] = df['route_distance'] * df['stops']        df['dist_squared'] = df['route_distance'] ** 2        df['stops_squared'] = df['stops'] ** 2        df['sqrt_distance'] = np.sqrt(df['route_distance'])        df['log_distance'] = np.log1p(df['route_distance'])        df['log_stops'] = np.log1p(df['stops'])                # 4. Efficiency scores (potential confounders)        print("4. Calculating efficiency indicators...")                # Theoretical minimum time (assuming optimal conditions)        base_velocity = 1.5  # m/s walking speed        stop_time = 15  # seconds per stop        df['theoretical_time'] = df['route_distance']/base_velocity + df['stops']*stop_time        df['efficiency_ratio'] = df['time_true'] / (df['theoretical_time'] + 1)                # 5. Outlier indicators        print("5. Identifying outliers...")                # Statistical outliers        for col in ['route_distance', 'stops', 'time_true']:            mean = df[col].mean()            std = df[col].std()            df[f'{col}_zscore'] = np.abs((df[col] - mean) / std)            df[f'is_{col}_outlier'] = (df[f'{col}_zscore'] > 3).astype(int)                print(f"\n✓ Created {len(df.columns) - len(data.columns)} new features")        print(f"Total features: {len(df.columns)}")                return df        def segment_analysis(self, data: pd.DataFrame):        """Analyze different operational segments"""        print("\n" + "="*60)        print("SEGMENT ANALYSIS")        print("="*60)                # Create segments        data['segment'] = 'Standard'        data.loc[(data['route_distance'] < 500) & (data['stops'] < 10), 'segment'] = 'Quick_Picks'        data.loc[(data['route_distance'] > 2500) & (data['stops'] > 25), 'segment'] = 'Long_Complex'        data.loc[(data['route_distance'] > 2500) & (data['stops'] <= 25), 'segment'] = 'Long_Simple'        data.loc[(data['route_distance'] <= 1000) & (data['stops'] > 25), 'segment'] = 'Short_Dense'                print("\nSegment Distribution:")        print(data['segment'].value_counts())                print("\nSegment Performance:")        segment_stats = data.groupby('segment').agg({            'route_distance': 'mean',            'stops': 'mean',             'time_true': 'mean',            'wave_id': 'count'        }).round(1)        segment_stats.columns = ['Avg_Distance', 'Avg_Stops', 'Avg_Time', 'Count']        print(segment_stats)                # Calculate time per meter and time per stop for each segment        print("\nSegment Efficiency:")        for segment in data['segment'].unique():            seg_data = data[data['segment'] == segment]            time_per_m = seg_data['time_true'].sum() / seg_data['route_distance'].sum()            time_per_stop = seg_data['time_true'].sum() / seg_data['stops'].sum()            print(f"  {segment:15} → {time_per_m:.3f} s/m, {time_per_stop:.1f} s/stop")                return data        def train_advanced_models(self, data: pd.DataFrame):        """Train and compare advanced models"""        print("\n" + "="*60)        print("ADVANCED MODEL TRAINING")        print("="*60)                # Prepare features        feature_cols = [            'route_distance', 'stops',            'stops_per_meter', 'meters_per_stop',            'total_complexity', 'dist_stops_interaction',            'dist_squared', 'stops_squared',            'sqrt_distance', 'log_distance', 'log_stops',            'is_long_route', 'high_stops'        ]                X = data[feature_cols]        y = data['time_true']                # Split data        X_train, X_test, y_train, y_test = train_test_split(            X, y, test_size=0.2, random_state=42        )                # Scale features for better performance        scaler = StandardScaler()        X_train_scaled = scaler.fit_transform(X_train)        X_test_scaled = scaler.transform(X_test)                models = {            'Random Forest': RandomForestRegressor(                n_estimators=200,                 max_depth=15,                min_samples_split=10,                min_samples_leaf=5,                random_state=42            ),            'Gradient Boosting': GradientBoostingRegressor(                n_estimators=200,                learning_rate=0.05,                max_depth=5,                min_samples_split=10,                random_state=42            ),            'RF with Tuning': RandomForestRegressor(                n_estimators=300,                max_depth=20,                min_samples_split=5,                min_samples_leaf=2,                max_features='sqrt',                random_state=42            )        }                results = {}                for name, model in models.items():            print(f"\nTraining {name}...")                        # Use scaled features for GB, raw for RF            if 'Gradient' in name:                model.fit(X_train_scaled, y_train)                y_pred = model.predict(X_test_scaled)            else:                model.fit(X_train, y_train)                y_pred = model.predict(X_test)                        # Calculate metrics            mae = mean_absolute_percentage_error(y_test, y_pred)            residuals = y_pred - y_test                        results[name] = {                'mae': mae,                'predictions': y_pred,                'residuals': residuals,                'model': model            }                        print(f"  MAE: {mae*100:.1f}%")            print(f"  Mean residual: {np.mean(residuals):.1f} sec")            print(f"  Std residual: {np.std(residuals):.1f} sec")                        # Feature importance for tree models            if hasattr(model, 'feature_importances_'):                importance = pd.DataFrame({                    'feature': feature_cols,                    'importance': model.feature_importances_                }).sort_values('importance', ascending=False)                print(f"\n  Top 5 Important Features:")                for idx, row in importance.head(5).iterrows():                    print(f"    {row['feature']:25} {row['importance']:.3f}")                return results, X_test, y_test        def segment_specific_models(self, data: pd.DataFrame):        """Train separate models for different segments"""        print("\n" + "="*60)        print("SEGMENT-SPECIFIC MODELS")        print("="*60)                data = self.segment_analysis(data)        segment_results = {}                for segment in data['segment'].unique():            if segment == 'Quick_Picks' and data[data['segment'] == segment].shape[0] < 100:                continue  # Skip if too few samples                            print(f"\nTraining model for: {segment}")            seg_data = data[data['segment'] == segment]                        if len(seg_data) < 50:                print(f"  Skipped - only {len(seg_data)} samples")                continue                        # Simple features for segment models            X = seg_data[['route_distance', 'stops']]            y = seg_data['time_true']                        X_train, X_test, y_train, y_test = train_test_split(                X, y, test_size=0.2, random_state=42            )                        model = RandomForestRegressor(                n_estimators=100,                max_depth=10,                random_state=42            )            model.fit(X_train, y_train)            y_pred = model.predict(X_test)                        mae = mean_absolute_percentage_error(y_test, y_pred)            segment_results[segment] = mae                        print(f"  Samples: {len(seg_data)}")            print(f"  MAE: {mae*100:.1f}%")                print("\n" + "-"*40)        print("Segment-Specific vs Global Model:")        avg_segment_mae = np.mean(list(segment_results.values()))        print(f"Average Segment MAE: {avg_segment_mae*100:.1f}%")                return segment_results        def diagnose_high_error_cases(self, results: dict, X_test, y_test):        """Identify patterns in high-error predictions"""        print("\n" + "="*60)        print("HIGH ERROR CASE ANALYSIS")        print("="*60)                best_model_name = min(results.keys(), key=lambda k: results[k]['mae'])        best_result = results[best_model_name]                # Create analysis dataframe        analysis = pd.DataFrame(X_test)        analysis['time_true'] = y_test        analysis['time_pred'] = best_result['predictions']        analysis['error_pct'] = 100 * (analysis['time_pred'] - analysis['time_true']) / analysis['time_true']        analysis['abs_error_pct'] = np.abs(analysis['error_pct'])                # Identify high error cases        high_error = analysis[analysis['abs_error_pct'] > 50]        medium_error = analysis[(analysis['abs_error_pct'] > 20) & (analysis['abs_error_pct'] <= 50)]        low_error = analysis[analysis['abs_error_pct'] <= 20]                print(f"\nError Distribution (using {best_model_name}):")        print(f"  Low error (≤20%):    {len(low_error)} cases ({len(low_error)/len(analysis)*100:.1f}%)")        print(f"  Medium error (20-50%): {len(medium_error)} cases ({len(medium_error)/len(analysis)*100:.1f}%)")        print(f"  High error (>50%):    {len(high_error)} cases ({len(high_error)/len(analysis)*100:.1f}%)")                if len(high_error) > 0:            print(f"\nHigh Error Case Characteristics:")            print(f"  Avg distance: {high_error['route_distance'].mean():.0f} m (overall: {analysis['route_distance'].mean():.0f} m)")            print(f"  Avg stops: {high_error['stops'].mean():.1f} (overall: {analysis['stops'].mean():.1f})")            print(f"  Avg true time: {high_error['time_true'].mean():.0f} s (overall: {analysis['time_true'].mean():.0f} s)")                        # Check if high errors are systematic            print(f"\nError Bias:")            print(f"  Mean error in high cases: {high_error['error_pct'].mean():.1f}%")            print(f"  (Positive = overestimation, Negative = underestimation)")                return analysis        def recommend_improvements(self, mae_results: dict):        """Provide specific recommendations based on results"""        print("\n" + "="*60)        print("RECOMMENDATIONS FOR ACHIEVING <15% MAE")        print("="*60)                best_mae = min(r['mae'] for r in mae_results.values()) * 100                if best_mae < 15:            print(f"✅ SUCCESS! Achieved {best_mae:.1f}% MAE")            print("\nTo maintain and improve:")            print("1. Regular recalibration with new data")            print("2. Monitor for drift in operational patterns")            print("3. Consider online learning for adaptation")        else:            gap = best_mae - 15            print(f"Current best: {best_mae:.1f}% MAE (gap: {gap:.1f}%)")                        print("\n📋 Priority Actions:")                        if gap > 20:                print("\n1. DATA QUALITY (High Priority)")                print("   - Verify time measurements are accurate")                print("   - Check for missing stops or merged waves")                print("   - Validate distance calculations")                print("   - Remove outliers and anomalous waves")                            print("\n2. MISSING FEATURES (Critical)")            print("   - Add: Item weight/volume per pick")            print("   - Add: Pick height/level (ground, middle, top)")            print("   - Add: Worker ID or experience level")            print("   - Add: Time of day / shift")            print("   - Add: Congestion metrics")            print("   - Add: Equipment type used")                        print("\n3. OPERATIONAL SEGMENTATION")            print("   - Separate models for:")            print("     • Single-item vs batch picking")            print("     • Different warehouse zones")            print("     • Peak vs off-peak hours")            print("     • Regular vs expedited orders")                        print("\n4. ADVANCED MODELING")            print("   - Try: Neural networks with embedding layers")            print("   - Try: XGBoost with careful hyperparameter tuning")            print("   - Try: Ensemble of specialized models")            print("   - Try: Time series features if temporal patterns exist")                        if best_mae > 30:                print("\n⚠️ CRITICAL ISSUE DETECTED")                print("   The error is too high for typical modeling fixes.")                print("   Investigate potential data collection issues:")                print("   - Are times recorded at correct granularity?")                print("   - Is there a delay between recorded and actual times?")                print("   - Are some operations missing from the time measurements?")        def load_base_data(self) -> pd.DataFrame:        """Load and merge base data"""        # Load routes for distance        routes = pd.read_feather(self.s2dir / "baseline_routes_bywave.feather")        dist_df = routes.groupby('wave_id')['route_distance'].max().reset_index()        dist_df['wave_id'] = dist_df['wave_id'].astype(str)                # Load metrics for time        metrics = pd.read_csv(self.s2dir / "baseline_metrics_bywave.csv")        time_col = 'time_vendor_throttle' if 'time_vendor_throttle' in metrics.columns else 'time_vendor'        time_df = metrics[['wave_id', time_col]].copy()        time_df.columns = ['wave_id', 'time_true']        time_df['wave_id'] = time_df['wave_id'].astype(str)                # Load picks for stops count        picks = pd.read_parquet(self.canon / "picks.parquet")        stops_df = picks.groupby('wave_id').size().reset_index(name='stops')        stops_df['wave_id'] = stops_df['wave_id'].astype(str)                # Merge all data        data = (dist_df                .merge(time_df, on='wave_id', how='inner')                .merge(stops_df, on='wave_id', how='inner'))                # Clean data        data = data[            (data['route_distance'] > 0) &             (data['time_true'] > 1) &             (data['stops'] > 0)        ].copy()                print(f"✓ Loaded {len(data)} waves with complete data")        return data        def run(self):        """Execute the advanced calibration pipeline"""        print("\n🚀 ADVANCED TWIN CALIBRATION PIPELINE")        print("="*60)                # Load base data        print("\nLoading data...")        data = self.load_base_data()                # Feature engineering        data_enhanced = self.engineer_features(data)                # Segment analysis        self.segment_analysis(data_enhanced)                # Train advanced models        results, X_test, y_test = self.train_advanced_models(data_enhanced)                # Segment-specific models        segment_results = self.segment_specific_models(data_enhanced)                # Diagnose errors        error_analysis = self.diagnose_high_error_cases(results, X_test, y_test)                # Recommendations        self.recommend_improvements(results)                print("\n" + "="*60)        print("✅ ADVANCED CALIBRATION COMPLETE")        print("="*60)                return results, error_analysis# Usageif __name__ == "__main__":    BASE_PATH = r"BASE"        calibration = AdvancedTwinCalibration(BASE_PATH)    results, analysis = calibration.run()

In [ ]:
#S4— Twin Calibration (Transparency)from __future__ import annotationsfrom pathlib import Pathimport json, re, mathimport numpy as npimport pandas as pdfrom sklearn.model_selection import train_test_splitfrom sklearn.ensemble import RandomForestRegressor# ---------------------------- Config ----------------------------BASE   = Path(r"BASE")CANON  = BASE / "canonical"S2DIR  = BASE / "results" / "baselines"DOPEN  = BASE / "Datasets" / "OpenFactoryTwin"CALDIR = DOPEN / "calibration"RSLT   = BASE / "results" / "cpsd"CALDIR.mkdir(parents=True, exist_ok=True)RSLT.mkdir(parents=True, exist_ok=True)MAE_REQ = 0.12  # ≤12% required for PASS (Nature-style gate)def _read_csv(path: Path) -> pd.DataFrame:    # robust CSV reader (auto-delimiter), no engine-specific options that break    return pd.read_csv(path, sep=None, engine="python", on_bad_lines="skip")def _find_time_col(cols) -> str|None:    # Prefer vendor/throttle columns; else first numeric 'time*'    prios = [        r"^time[_\s]*vendor[_\s]*throttle$",        r"^time[_\s]*vendor$",        r"^time[_\s]*(sec|seconds)?$",        r"^time$",    ]    lc = {str(c).strip().lower(): c for c in cols}    for pat in prios:        for k,v in lc.items():            if re.search(pat, k): return v    # fallback: first column whose name starts with "time"    for k,v in lc.items():        if k.startswith("time"): return v    return Nonedef _ensure_wave_col(df: pd.DataFrame, hint: str) -> str:    # find a wave-like column name    for c in df.columns:        if re.search(r"\bwave(_?id)?\b|wavenumber|batch", str(c), re.I):             return c    raise RuntimeError(        f"{hint} is missing a wave identifier column (e.g., 'wave_id'). "        "Re-run S2 so metrics include per-wave rows with a wave_id."    )def _degeneracy_check(time_s: np.ndarray, dist_m: np.ndarray) -> tuple[bool,float,float,float]:    """Check if time ≈ k*distance (constant speed identity). Returns (degenerate, k, r2_origin, cv_ratio)."""    t = np.asarray(time_s, float); d = np.asarray(dist_m, float)    m = np.isfinite(t)&np.isfinite(d)&(t>0)&(d>0)    if not np.any(m):         return False, float("nan"), float("nan"), float("nan")    t = t[m]; d = d[m]    k = (d*t).sum() / (d*d).sum()  # slope through origin    pred = k*d    ss_res = np.sum((t - pred)**2)    ss_tot = np.sum((t - np.mean(t))**2)    r2 = 1 - ss_res/ss_tot if ss_tot>0 else float("nan")    ratio = t/d    cv = float(np.std(ratio)/np.mean(ratio)) if np.mean(ratio)>0 else float("nan")    deg = (np.isfinite(r2) and r2>0.999) and (np.isfinite(cv) and cv<0.02)    return deg, float(k), float(r2), float(cv)def _mape(y_true, y_pred) -> float:    y_true = np.asarray(y_true, float); y_pred = np.asarray(y_pred, float)    m = np.isfinite(y_true)&np.isfinite(y_pred)&(y_true>1e-9)    if not np.any(m): return float("nan")    return float(np.mean(np.abs((y_pred[m]-y_true[m]) / y_true[m])))def _bland_altman(y_true, y_pred) -> tuple[float,float,float]:    diff = np.asarray(y_pred, float) - np.asarray(y_true, float)    diff = diff[np.isfinite(diff)]    bias = float(np.mean(diff)) if diff.size else float("nan")    sd   = float(np.std(diff, ddof=1)) if diff.size>1 else float("nan")    loa_low, loa_high = bias - 1.96*sd, bias + 1.96*sd    return bias, loa_low, loa_highprint("=== S4 — Twin Calibration (planning-time) ===")# ---------------------------- Load S2/S1 inputs ----------------------------routes_p  = S2DIR / "baseline_routes_bywave.feather"metrics_p = S2DIR / "baseline_metrics_bywave.csv"picks_p   = CANON / "picks.parquet"if not routes_p.exists():    raise FileNotFoundError(f"Missing {routes_p} (run S2).")if not metrics_p.exists():    raise FileNotFoundError(f"Missing {metrics_p} (run S2).")if not picks_p.exists():    raise FileNotFoundError(f"Missing {picks_p} (run S1).")routes  = pd.read_feather(routes_p)metrics = _read_csv(metrics_p)picks   = pd.read_parquet(picks_p)# distance per waveif "route_distance" not in routes.columns:    # try common alternates    cand = [c for c in routes.columns if re.search(r"route[_\s]*distance|distance(_m)?|route[_\s]*len", str(c), re.I)]    if not cand:        raise RuntimeError("No distance column in routes (expected 'route_distance').")    routes["route_distance"] = routes[cand[0]]wave_routes_col = _ensure_wave_col(routes, "routes")wave_metrics_col = _ensure_wave_col(metrics, "metrics")dist_df = routes[[wave_routes_col, "route_distance"]].copy().rename(columns={wave_routes_col:"wave_id"})# A wave can have multiple route rows; take max or sum — you used max in earlier steps; keep consistent:dist_df = dist_df.groupby("wave_id", as_index=False)["route_distance"].max()dist_df["wave_id"] = dist_df["wave_id"].astype(str)# time from metricstime_col = _find_time_col(metrics.columns)if time_col is None:    raise RuntimeError("metrics has no usable time column (e.g., time_vendor_throttle / time_vendor / time).")time_df = metrics[[wave_metrics_col, time_col]].copy().rename(columns={wave_metrics_col:"wave_id", time_col:"time_true"})time_df["wave_id"] = time_df["wave_id"].astype(str)time_df["time_true"] = pd.to_numeric(time_df["time_true"], errors="coerce")# stops from picksstops_df = picks.groupby("wave_id").size().reset_index(name="stops")stops_df["wave_id"] = stops_df["wave_id"].astype(str)# Merge & cleandata = (dist_df.merge(time_df, on="wave_id", how="inner")               .merge(stops_df, on="wave_id", how="inner"))data = data.replace([np.inf,-np.inf], np.nan).dropna(subset=["route_distance","time_true","stops"])data = data[(data["route_distance"]>0) & (data["time_true"]>1) & (data["stops"]>0)].copy()n_all = len(data)if n_all < 200:    raise RuntimeError(f"Only {n_all} waves with complete data; need ≥200 to calibrate robustly. Check S1/S2 outputs.")print(f"[info] candidate waves: {n_all}")# ---------------------------- Degeneracy (leakage) check ----------------------------deg, k_slope, r2o, cv_ratio = _degeneracy_check(data["time_true"].to_numpy(), data["route_distance"].to_numpy())print(f"[guard] time≈k*distance check → degenerate={deg} | k={k_slope:.3f} s/m | R^2_origin={r2o:.6f} | CV(ratio)={cv_ratio:.4f}")if deg:    # This means metrics time is effectively computed from distance via constant speed for this baseline.    # You can still *use* this for planning-time visualization, but we won't call this a calibration.    outcsv = CALDIR / "combined_calibration.csv"    pd.DataFrame(columns=["time_true","time_sim"]).to_csv(outcsv, index=False, encoding="utf-8-sig")    meta = {        "utc": pd.Timestamp.utcnow().isoformat(),        "source": "baseline_planning_time",        "status": "FAIL (degenerate)",        "reason": "metrics time is essentially k*distance (constant-speed identity); need independent timestamps (telemetry or WMS wave times).",        "k_slope_s_per_m": k_slope,        "r2_origin": r2o,        "cv_ratio": cv_ratio,        "pairs": 0,        "combined_csv": outcsv.as_posix()    }    with open(RSLT / "twin_calibration.json","w",encoding="utf-8") as f: json.dump(meta, f, indent=2)    # write MD    md = RSLT / "S4_calibration_report.md"    md.write_text(        "### S4 — Twin Calibration (planning-time)\n\n"        f"**FAIL (degenerate)**: time is essentially `k·distance` for this baseline.\n\n"        f"- k (s/m): {k_slope:.3f}\n- R² through origin: {r2o:.6f}\n- CV(time/distance): {cv_ratio:.4f}\n\n"        "Provide independent timestamps (telemetry `x,y,t` or WMS wave start/finish) and re-run.\n",        encoding="utf-8"    )    raise SystemExit(0)# ---------------------------- Feature set & split ----------------------------df = data.copy()# simple, stable features (avoid anything derived from time_true)df["dist_squared"] = df["route_distance"]**2df["sqrt_distance"] = np.sqrt(df["route_distance"])df["log_distance"]  = np.log1p(df["route_distance"])df["stops_squared"] = df["stops"]**2df["dist_stops_interaction"] = df["route_distance"]*df["stops"]df["is_long_route"] = (df["route_distance"] > df["route_distance"].quantile(0.75)).astype(int)df["high_stops"]    = (df["stops"] > df["stops"].quantile(0.75)).astype(int)features = [    "route_distance","stops",    "dist_squared","stops_squared",    "sqrt_distance","log_distance","dist_stops_interaction",    "is_long_route","high_stops"]X = df[features].valuesy = df["time_true"].values# stratified-ish split by distance quantiles (to avoid distribution shift)q = pd.qcut(df["route_distance"], q=10, labels=False, duplicates="drop")X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=q)# ---------------------------- Train & evaluate ----------------------------rf = RandomForestRegressor(    n_estimators=300,    max_depth=20,    min_samples_split=5,    min_samples_leaf=2,    max_features="sqrt",    random_state=42,    n_jobs=-1)rf.fit(X_train, y_train)y_pred = rf.predict(X_test)mape = _mape(y_test, y_pred)bias, loa_low, loa_high = _bland_altman(y_test, y_pred)pass_gate = (np.isfinite(mape) and mape <= MAE_REQ)# ---------------------------- Write artifacts ----------------------------outcsv = CALDIR / "combined_calibration.csv"pd.DataFrame({"time_true": y_test, "time_sim": y_pred}).to_csv(outcsv, index=False, encoding="utf-8-sig")meta = {    "utc": pd.Timestamp.utcnow().isoformat(),    "type": "baseline_planning_time",    "status": "PASS" if pass_gate else "FAIL",    "mae_mape": float(mape),    "bias_seconds": float(bias),    "loa_low_seconds": float(loa_low),    "loa_high_seconds": float(loa_high),    "n_train": int(len(y_train)),    "n_eval": int(len(y_test)),    "features": features,    "model": "RandomForestRegressor",    "hyperparams": {        "n_estimators": 300,        "max_depth": 20,        "min_samples_split": 5,        "min_samples_leaf": 2,        "max_features": "sqrt",        "random_state": 42    },    "degeneracy_check": {        "k_slope_s_per_m": k_slope,        "r2_origin": r2o,        "cv_ratio": cv_ratio,        "degenerate": False    },    "combined_csv": outcsv.as_posix(),    "notes": "Calibrated to vendor/planning time from S2 metrics; not physical telemetry. Use for ε guardrails & Pareto; PSD claims remain conformal/ISO-based."}with open(RSLT / "twin_calibration.json","w",encoding="utf-8") as f:    json.dump(meta, f, indent=2)# Human-readable reportmd = RSLT / "S4_calibration_report.md"md.write_text(    "### S4 — Twin Calibration (planning-time)\n\n"    f"- **Target**: baseline-derived wave time (metrics column: `{_find_time_col(metrics.columns)}`)\n"    "- **Scope**: planning-time twin (not telemetry). Use for ε guardrails & Pareto only.\n"    f"- **Waves used**: total={n_all}, eval={len(y_test)}\n"    f"- **MAPE**: {mape*100:.2f}%  (gate ≤ {MAE_REQ*100:.0f}%)\n"    f"- **Bias**: {bias:.2f} s; **LoA**: [{loa_low:.2f}, {loa_high:.2f}] s\n"    f"- **Degeneracy check**: k={k_slope:.3f} s/m, R²₀={r2o:.6f}, CV={cv_ratio:.4f} → OK\n"    f"- **Features**: {', '.join(features)}\n"    "- **Model**: RandomForestRegressor (see JSON for hyperparams)\n"    "- **Artifacts**:\n"    f"  - `calibration/combined_calibration.csv` (hold-out: time_true,time_sim)\n"    f"  - `results/cpsd/twin_calibration.json`\n",    encoding="utf-8")banner = "[PASS]" if pass_gate else "[FAIL]"print(f"{banner} Twin calibration — MAPE={mape*100:.2f}%  (bias={bias:.2f}s, LoA=[{loa_low:.2f},{loa_high:.2f}]s)")print("CSV  →", outcsv.as_posix())print("JSON →", (RSLT/"twin_calibration.json").as_posix())print("MD   →", md.as_posix())if not pass_gate:    raise AssertionError("S4 gate failed: MAPE > 12%. See S4_calibration_report.md for details.")

In [ ]:
# S3 — Option B+ (EHB-min under time guard, S4/S5-compatible)from __future__ import annotationsfrom pathlib import Pathfrom dataclasses import dataclassimport numpy as npimport pandas as pd# ---------------- Paths ----------------BASE  = Path(r"BASE")CANON = BASE / "canonical"BOUT  = BASE / "results" / "baselines"HOUT  = BASE / "results" / "hrc"HOUT.mkdir(parents=True, exist_ok=True)picks_p   = CANON / "picks.parquet"layout_p  = CANON / "layout.parquet"broutes_p = BOUT / "baseline_routes_bywave.feather"tbase_p   = BOUT / "T_base_bywave.feather"hrc_routes_in  = HOUT / "hrc_routes_bywave.feather"   # input candidateshrc_metrics_in = HOUT / "hrc_metrics_bywave.csv"       # input for eps discoveryhrc_routes_out = HOUT / "hrc_routes_bywave.feather"    # overwrite with selectedhrc_metrics_out= HOUT / "hrc_metrics_bywave.csv"ehb_comp_out   = HOUT / "ehb_components.feather"# ---------------- Config ----------------@dataclassclass Cfg:    v_base: float = 1.0          # m/s → seconds via distance / v_base    burst_k: int = 3             # consecutive "high" picks before burst starts    w_reach_low: float = 0.2    w_reach_mid: float = 1.2    w_reach_high: float = 3.0    w_burst: float = 1.0    w_transition: float = 0.5    eps_allow: tuple = (0.0, 0.005, 0.01, 0.02)   # we’ll intersect with discovered set    tol_sec: float = 1e-6        # numerical tolerance on guard (very tight)    micro_max_iters: int = 25    # limit micro-optim passes per route    micro_window: int = 4        # local window for swap candidates    max_candidates: int = 12     # hard cap to avoid explosion (baseline algos + HRC)cfg = Cfg()# ---------------- Helpers ----------------def _read_feather(p: Path) -> pd.DataFrame:    if not p.exists(): return pd.DataFrame()    return pd.read_feather(p)def _read_parquet(p: Path) -> pd.DataFrame:    if not p.exists(): return pd.DataFrame()    return pd.read_parquet(p)def _read_csv(p: Path) -> pd.DataFrame:    if not p.exists(): return pd.DataFrame()    return pd.read_csv(p)def _safe_eps_col(df: pd.DataFrame) -> pd.Series:    # normalize epsilon column (float) if present    if "eps" in df.columns: return pd.to_numeric(df["eps"], errors="coerce")    if "epsilon" in df.columns: return pd.to_numeric(df["epsilon"], errors="coerce").rename("eps")    return pd.Series([np.nan]*len(df), name="eps")def _route_time_from_xy(x: np.ndarray, y: np.ndarray, v: float) -> float:    if x.size < 2: return 0.0    dist = np.hypot(np.diff(x), np.diff(y)).sum()    return float(dist / max(v, 1e-9))def _recompute_route_fields(df: pd.DataFrame) -> tuple[float, float, np.ndarray]:    """Return (route_distance, route_time, seg_len array) from x,y."""    if df.shape[0] == 0:        return 0.0, 0.0, np.array([0.0], dtype=float)    x = df["x"].to_numpy(float, copy=False)    y = df["y"].to_numpy(float, copy=False)    seg = np.r_[0.0, np.hypot(np.diff(x), np.diff(y))]    dist = float(seg.sum())    return dist, dist / cfg.v_base, segdef _infer_zone_thresholds(zs: pd.Series) -> tuple[float,float]:    zs = pd.to_numeric(zs, errors="coerce").dropna()    if zs.empty: return (0.33, 0.66)  # dummy cutpoints    q = zs.quantile([0.33, 0.66]).to_numpy()    return float(q[0]), float(q[1])def _zone_from_z(z: float, zlo: float, zmid: float) -> str:    if pd.isna(z): return "mid"    if z >= zmid: return "high"    if z >= zlo:  return "mid"    return "low"def _compute_ehb_components(route_df: pd.DataFrame, zlo: float, zmid: float) -> pd.DataFrame:    r = route_df.copy()    if "z" not in r.columns: r["z"] = np.nan    zones = r["z"].apply(lambda z: _zone_from_z(z, zlo, zmid)).astype("category")    reach = zones.map({"low":cfg.w_reach_low,"mid":cfg.w_reach_mid,"high":cfg.w_reach_high}).astype(float).to_numpy()    # burst (starting at k-th consecutive high)    hi = (zones == "high").to_numpy()    burst = np.zeros_like(hi, dtype=float)    run = 0    for i, v in enumerate(hi):        run = run + 1 if v else 0        if v and run >= cfg.burst_k:            burst[i] = cfg.w_burst * (run - cfg.burst_k + 1)    # transitions (zone changes)    trans = np.zeros_like(hi, dtype=float)    if len(hi) > 1:        trans[1:] = (zones.to_numpy()[1:] != zones.to_numpy()[:-1]) * cfg.w_transition    out = r[["wave_id","eps","algo","order_index","sku","bin_id","x","y","z"]].copy()    out["zone"] = zones.astype(str)    out["reach_pen"] = reach    out["burst_pen"] = burst    out["transition_pen"] = trans    out["ehb_pick"] = reach + burst + trans    return outdef _ehb_sum(route_df: pd.DataFrame, zlo: float, zmid: float) -> tuple[float, pd.DataFrame]:    comp = _compute_ehb_components(route_df, zlo, zmid)    return float(comp["ehb_pick"].sum()), compdef _micro_swap_min_ehb(route_df: pd.DataFrame, t_guard: float, zlo: float, zmid: float) -> pd.DataFrame:    """Greedy, local swaps to break long 'high' runs; keep guard. Small & safe."""    r = route_df.reset_index(drop=True).copy()    best_ehb, _ = _ehb_sum(r, zlo, zmid)    best = r.copy()    iters = 0    while iters < cfg.micro_max_iters:        iters += 1        improved = False        zones = best.get("z", pd.Series([np.nan]*len(best))).apply(lambda z: _zone_from_z(z, zlo, zmid)).to_numpy()        # find start indices of 'high' runs        idxs = [i for i in range(len(best)) if zones[i] == "high"]        if not idxs:            break        i0 = None        # locate a run of length >= burst_k        run = 0        for j in range(len(best)):            run = run + 1 if zones[j]=="high" else 0            if run >= cfg.burst_k:                i0 = j - run + 1                break        if i0 is None:            break        # try to pull a non-high within a small window to break the run        j_candidates = range(max(0, i0 - cfg.micro_window), min(len(best), i0 + cfg.micro_window + 1))        # prefer nearest non-high        for j in j_candidates:            if zones[j] != "high":                cand = best.copy()                # swap positions i0 and j (or move j next to i0)                cand.iloc[[i0, j]] = cand.iloc[[j, i0]].to_numpy()                # recompute time                dist, t, seg = _recompute_route_fields(cand)                if t <= t_guard + cfg.tol_sec:                    ehb_cand, _ = _ehb_sum(cand, zlo, zmid)                    if ehb_cand + 1e-9 < best_ehb:                        best = cand                        best["route_distance"] = dist                        best["route_time"] = t                        best["seg_len"] = seg                        best["order_index"] = np.arange(len(best), dtype=int)                        best_ehb = ehb_cand                        improved = True                        break        if not improved:            break    return best# ---------------- Load inputs ----------------picks   = _read_parquet(picks_p)layout  = _read_parquet(layout_p)broutes = _read_feather(broutes_p)tbase   = _read_feather(tbase_p)hrc_r   = _read_feather(hrc_routes_in)hrc_m   = _read_csv(hrc_metrics_in)# Basic checksfor pth, df in [(picks_p,picks),(layout_p,layout),(broutes_p,broutes),(tbase_p,tbase)]:    if df.empty:        raise FileNotFoundError(f"Missing or empty: {pth}")# Normalize keys / columnsdef _as_str(s): return s.astype(str)if "wave_id" in picks.columns: picks["wave_id"] = _as_str(picks["wave_id"])if "wave_id" in layout.columns and layout["bin_id"].dtype != object:    layout["bin_id"] = layout["bin_id"].astype(str)for df in (broutes, hrc_r):    if "wave_id" in df.columns: df["wave_id"] = _as_str(df["wave_id"])    if "algo" not in df.columns: df["algo"] = "unknown"    if "eps" not in df.columns:        df["eps"] = _safe_eps_col(df)    else:        df["eps"] = pd.to_numeric(df["eps"], errors="coerce")if "wave_id" in tbase.columns: tbase["wave_id"] = _as_str(tbase["wave_id"])if "T_base" not in tbase.columns:    # try common fallbacks    for c in ["T_base", "Tbase", "time_base", "t_base"]:        if c in tbase.columns:            tbase = tbase.rename(columns={c:"T_base"}); breakif "T_base" not in tbase.columns:    raise RuntimeError("T_base column not found in T_base_bywave.feather.")# Compose eps seteps_discovered = set(pd.to_numeric(_safe_eps_col(hrc_m), errors="coerce").dropna().unique())eps_discovered |= set(pd.to_numeric(_safe_eps_col(hrc_r), errors="coerce").dropna().unique())eps_set = sorted(e for e in cfg.eps_allow if e in eps_discovered or e in (0.0,))  # always include 0.0if not eps_set:    eps_set = list(cfg.eps_allow)# Precompute z cut points (for zone)zlo, zmid = _infer_zone_thresholds(picks["z"] if "z" in picks.columns else pd.Series([], dtype=float))# Ensure z for baselines: join z by bin_id if baseline lacks zneed_z = ("z" not in broutes.columns) or broutes["z"].isna().all()if need_z and ("bin_id" in broutes.columns) and ("z" in layout.columns):    lay_z = layout[["bin_id","z"]].drop_duplicates("bin_id")    lay_z["bin_id"] = lay_z["bin_id"].astype(str)    broutes = broutes.merge(lay_z, on="bin_id", how="left")# Build baseline candidate index per wave: keep all algos presentif "algo" not in broutes.columns:    broutes["algo"] = "baseline"b_meta = (broutes          .groupby(["wave_id","algo"], as_index=False)["route_time"].min())# Build quick access dictsTb = dict(zip(tbase["wave_id"], pd.to_numeric(tbase["T_base"], errors="coerce").fillna(0.0)))waves = sorted(set(broutes["wave_id"]).intersection(set(tbase["wave_id"])))# Assemble and selectsel_routes = []     # concatenated final routessel_metrics = []    # per (wave, eps)sel_comps = []      # per-pick componentsfor e in eps_set:    e = float(e)    t_mult = 1.0 + e    # HRC candidates for this eps (may be empty)    hrc_e = hrc_r[pd.to_numeric(hrc_r["eps"], errors="coerce").fillna(-1).eq(e)].copy()    # For each wave    for w in waves:        tbase_w = Tb.get(w, np.nan)        if not np.isfinite(tbase_w):  # skip if no base            continue        t_guard = t_mult * tbase_w + cfg.tol_sec        # ---- Build candidate list (DataFrame per candidate) ----        candidates = []        # Baseline algos available for this wave        bros = broutes[broutes["wave_id"] == w]        for algo_name, g in bros.groupby("algo"):            c = g.sort_values("order_index").copy()            # recompute distance/time from x,y to be consistent            dist, t, seg = _recompute_route_fields(c)            c["route_distance"] = dist            c["route_time"] = t            c["seg_len"] = seg            c["eps"] = e            c["algo"] = f"baseline_{algo_name}"            if t <= t_guard:                candidates.append(c)            if len(candidates) >= cfg.max_candidates: break        # HRC candidate (if any exists for this wave, eps)        he = hrc_e[hrc_e["wave_id"] == w].sort_values("order_index")        if not he.empty:            c = he.copy()            # ensure numeric & recompute            dist, t, seg = _recompute_route_fields(c)            c["route_distance"] = dist            c["route_time"] = t            c["seg_len"] = seg            c["eps"] = e            # keep its algo label if present; else tag            if "algo" not in c.columns or c["algo"].isna().all():                c["algo"] = f"hrc_eps_{int(round(e*1000))/10.0:g}"            if t <= t_guard:                candidates.append(c)        if not candidates:            # No guard-satisfying candidate; fall back to min-time baseline (then mark guard_ok False)            if bros.empty:                continue            c0 = (bros.sort_values(["route_time","order_index"])                     .copy())            dist, t, seg = _recompute_route_fields(c0)            c0["route_distance"] = dist            c0["route_time"] = t            c0["seg_len"] = seg            c0["eps"] = e            c0["algo"] = "baseline_fallback"            chosen = c0            guard_ok = False            # EHB            ehb_sum, comp = _ehb_sum(chosen, zlo, zmid)        else:            # Evaluate EHB on each candidate            scored = []            for cand in candidates[:cfg.max_candidates]:                ehb_sum, _ = _ehb_sum(cand, zlo, zmid)                scored.append((ehb_sum, cand))            # micro-optimization on the best few (2) to squeeze extra EHB            scored.sort(key=lambda x: x[0])            improved = []            for k in range(min(2, len(scored))):                base_ehb, cand = scored[k]                # try micro swaps                tuned = _micro_swap_min_ehb(cand, t_guard, zlo, zmid)                ehb_tuned, _ = _ehb_sum(tuned, zlo, zmid)                improved.append((ehb_tuned, tuned))            improved.sort(key=lambda x: x[0])            ehb_best, chosen = improved[0]            guard_ok = chosen["route_time"].iloc[0] <= t_guard + cfg.tol_sec            ehb_sum, comp = _ehb_sum(chosen, zlo, zmid)        # Prepare outputs for this (w,e)        chosen = chosen.reset_index(drop=True).copy()        # standardize columns & order_index        chosen["wave_id"] = w        chosen["eps"] = e        chosen["order_index"] = np.arange(len(chosen), dtype=int)        # stability: ensure columns exist        for col in ["sku","bin_id","z","algo","x","y","seg_len","route_distance","route_time"]:            if col not in chosen.columns:                chosen[col] = np.nan        # changed_route flag vs the min-time baseline at eps=0 (reference)        # build reference (baseline min-time at eps=0) once via bros        ref = (bros.sort_values(["route_time","order_index"])                   .reset_index(drop=True)[["bin_id","x","y"]])        seq_ref = ref.to_numpy() if not ref.empty else None        seq_cur = chosen[["bin_id","x","y"]].to_numpy()        changed = True        if seq_ref is not None and seq_ref.shape == seq_cur.shape:            changed = not np.allclose(seq_ref, seq_cur, equal_nan=True)        # metrics row        dist, t, seg = _recompute_route_fields(chosen)        metrics_row = {            "wave_id": w,            "eps": e,            "guard_ok": bool(guard_ok),            "route_distance": float(dist),            "time_route": float(t),            "T_base": float(tbase_w),            "time_guard": float(t_guard),            "changed_route": bool(changed),        }        # components DF        comp = _compute_ehb_components(chosen, zlo, zmid)        comp["eps"] = e        comp["algo"] = chosen["algo"].iloc[0] if "algo" in chosen.columns else "unknown"        # aggregates        metrics_row["reach_sum"] = float(comp["reach_pen"].sum())        metrics_row["burst_sum"] = float(comp["burst_pen"].sum())        metrics_row["trans_sum"] = float(comp["transition_pen"].sum())        metrics_row["EHB"]       = float(comp["ehb_pick"].sum())        # collect        sel_routes.append(chosen)        sel_metrics.append(metrics_row)        sel_comps.append(comp)# ---------------- Persist ----------------if sel_routes:    routes_out = pd.concat(sel_routes, ignore_index=True)else:    routes_out = pd.DataFrame(columns=["wave_id","eps","algo","order_index","sku","bin_id","x","y","z","seg_len","route_distance","route_time"])metrics_out = pd.DataFrame(sel_metrics)# keep column names compatible with your downstream S5metrics_out = metrics_out[[    "wave_id","eps","guard_ok","route_distance","time_route","T_base","time_guard",    "reach_sum","burst_sum","trans_sum","EHB","changed_route"]]ehb_out = pd.concat(sel_comps, ignore_index=True) if sel_comps else pd.DataFrame(    columns=["wave_id","eps","algo","order_index","sku","bin_id","x","y","z","zone","reach_pen","burst_pen","transition_pen","ehb_pick"])# writeroutes_out.reset_index(drop=True).to_feather(hrc_routes_out)metrics_out.to_csv(hrc_metrics_out, index=False, encoding="utf-8-sig")ehb_out.reset_index(drop=True).to_feather(ehb_comp_out)# ---------------- Diagnostics ----------------def _pct(x): return f"{100.0*float(x):.2f}%"guard_by_eps = (metrics_out                .groupby("eps", as_index=False)["guard_ok"]                .mean())chg_by_eps = (metrics_out              .groupby("eps", as_index=False)["changed_route"]              .mean())print("=== S3 — Option B+ (EHB-min under guard) — COMPLETE ===")print(f"waves: {len(waves)} | eps set: {tuple(eps_set)} | v_base={cfg.v_base} m/s")print("Guard OK by ε:", {f"{int(e*100)}%": _pct(guard_by_eps[guard_by_eps['eps']==e]['guard_ok'].iloc[0] if not guard_by_eps[guard_by_eps['eps']==e].empty else 0.0) for e in eps_set})print("Changed-route share by ε:", {f"{int(e*100)}%": _pct(chg_by_eps[chg_by_eps['eps']==e]['changed_route'].iloc[0] if not chg_by_eps[chg_by_eps['eps']==e].empty else 0.0) for e in eps_set})print("\nFiles written:")print(" -", hrc_routes_out)print(" -", hrc_metrics_out)print(" -", ehb_comp_out)

In [ ]:
# S5 — Iso-time Pareto & Robustness (ΔEHB under ε-guard)from __future__ import annotationsfrom pathlib import Pathimport numpy as np, pandas as pd, math, json, textwrapimport matplotlib.pyplot as pltfrom datetime import datetime# ----------------------------- Config ----------------------------------------BASE   = Path(r"BASE")HOUT   = BASE / "results" / "hrc"S5DIR  = BASE / "results" / "s5_headroom"S5DIR.mkdir(parents=True, exist_ok=True)METRICS_P = HOUT / "hrc_metrics_bywave.csv"COMP_P    = HOUT / "ehb_components.feather"   # optional but recommended# Iso-time slack γ (absolute additive over ε)GAMMA_DEFAULT = 0.002     # 0.2%GAMMA_SET     = [0.001, 0.002, 0.003]   # 0.1%, 0.2%, 0.3%# Consider these ε values (others in file will be auto-included)EPS_CANON = [0.005, 0.01, 0.02]  # 0.5%, 1.0%, 2.0%EPS_ECDFS = True                 # whether to export ECDF CSVs/plots# BootstrapN_BOOT   = 5000SEED     = 42# Weight sensitivity (only if components exist)DO_WEIGHT_SENS = TrueBURST_K        = 3  # must match S3# Weight grids to test (reach_low, reach_mid, reach_high, burst, transition)WEIGHT_GRID = [    (0.2, 1.2, 3.0, 1.0, 0.5),   # default (S3)    (0.2, 1.0, 4.0, 1.0, 0.5),   # heavier high reach    (0.1, 0.8, 3.0, 1.0, 0.5),   # lighter overall reach    (0.2, 1.2, 3.0, 1.5, 0.5),   # stronger burst penalty    (0.2, 1.2, 3.0, 1.0, 0.8),   # stronger transition penalty]# Gates (informative; not hard failures)TARGET_EPS_FOR_GATE = 0.02   # evaluate gate at 2.0%TARGET_MIN_DELTA    = -2.0   # want median ΔEHB% ≤ -2.0 at target ε# -------------------------- Helpers ------------------------------------------def _fmt_pct(x):    try:        return f"{100.0*float(x):.2f}%"    except:        return "NA"def _bootstrap_ci_median(x: np.ndarray, n_boot=5000, seed=42, ci=0.95):    rng = np.random.default_rng(seed)    x = np.asarray(x, float)    x = x[np.isfinite(x)]    if x.size == 0:        return (np.nan, np.nan)    idx = rng.integers(0, x.size, size=(n_boot, x.size))    med = np.median(x[idx], axis=1)    lo = np.percentile(med, (1-ci)/2*100)    hi = np.percentile(med, (1+ci)/2*100)    return float(lo), float(hi)def _ecdf(arr):    arr = np.asarray(arr, float)    arr = arr[np.isfinite(arr)]    if arr.size == 0:        return np.array([]), np.array([])    x = np.sort(arr)    y = np.arange(1, x.size+1)/x.size    return x, ydef _ensure_cols(df: pd.DataFrame, cols):    missing = [c for c in cols if c not in df.columns]    if missing:        raise RuntimeError(f"Missing columns: {missing}")def _load_metrics(path: Path) -> pd.DataFrame:    if not path.exists():        raise FileNotFoundError(f"Missing: {path}")    df = pd.read_csv(path)    # canonicalize    if "wave_id" not in df.columns:        # try best-effort        for c in df.columns:            if str(c).lower().strip() in ("wave","waveid","wave_id","batch","wavenumber"):                df = df.rename(columns={c:"wave_id"})                break    _ensure_cols(df, ["wave_id","eps","EHB","time_route","T_base"])    # numeric casts    df["wave_id"]    = df["wave_id"].astype(str)    df["eps"]        = pd.to_numeric(df["eps"], errors="coerce")    df["EHB"]        = pd.to_numeric(df["EHB"], errors="coerce")    df["time_route"] = pd.to_numeric(df["time_route"], errors="coerce")    df["T_base"]     = pd.to_numeric(df["T_base"], errors="coerce")    if "changed_route" in df.columns:        df["changed_route"] = df["changed_route"].astype(bool)    else:        df["changed_route"] = False    return dfdef _derive_delta_table(metrics: pd.DataFrame, eps_list: list[float], gamma: float) -> pd.DataFrame:    """Return table with ΔEHB% and filters applied for each eps in eps_list."""    # iso-time guard recompute: d_time ≤ ε + γ    m = metrics.copy()    m["d_time"] = (m["time_route"] - m["T_base"]) / m["T_base"]    # baseline per wave (eps==0)    base = (m[m["eps"].fillna(0.0).eq(0.0)]            .loc[:, ["wave_id","EHB","time_route","T_base"]]            .rename(columns={"EHB":"EHB0","time_route":"time0","T_base":"T_base0"}))    # join for eps>0    out = []    for e in eps_list:        sub = m[m["eps"].round(6).eq(round(e,6))].copy()        sub = sub.merge(base, on="wave_id", how="inner")        sub["guard_ok_new"] = sub["d_time"] <= (e + gamma + 1e-12)        # ΔEHB% w.r.t. eps=0        sub["delta_ehb_pct"] = 100.0 * (sub["EHB"] - sub["EHB0"]) / sub["EHB0"]        sub["eps"] = float(e)        out.append(sub)    if not out:        return pd.DataFrame(columns=["wave_id","eps","delta_ehb_pct","changed_route","guard_ok_new"])    return pd.concat(out, ignore_index=True)def _summarize_by_eps(delta_df: pd.DataFrame) -> pd.DataFrame:    rows = []    for e, g in delta_df.groupby("eps", dropna=True):        g_guard = g[g["guard_ok_new"]]        n_total = len(g_guard)        if n_total == 0:            rows.append({                "eps": e, "n_total": 0, "n_changed": 0, "changed_share": np.nan,                "median_delta_changed": np.nan, "ci_lo_changed": np.nan, "ci_hi_changed": np.nan,                "median_delta_all": np.nan, "ci_lo_all": np.nan, "ci_hi_all": np.nan            })            continue        # changed-only        ch = g_guard[g_guard["changed_route"]]        med_c = np.median(ch["delta_ehb_pct"]) if len(ch) else np.nan        lo_c, hi_c = _bootstrap_ci_median(ch["delta_ehb_pct"].to_numpy(), N_BOOT, SEED) if len(ch) else (np.nan,np.nan)        # all waves        med_a = np.median(g_guard["delta_ehb_pct"])        lo_a, hi_a = _bootstrap_ci_median(g_guard["delta_ehb_pct"].to_numpy(), N_BOOT, SEED)        rows.append({            "eps": e,            "n_total": int(n_total),            "n_changed": int(ch.shape[0]),            "changed_share": float(ch.shape[0]/n_total) if n_total else np.nan,            "median_delta_changed": float(med_c),            "ci_lo_changed": float(lo_c),            "ci_hi_changed": float(hi_c),            "median_delta_all": float(med_a),            "ci_lo_all": float(lo_a),            "ci_hi_all": float(hi_a)        })    return pd.DataFrame(rows).sort_values("eps")def _save_csv(df: pd.DataFrame, name: str) -> Path:    p = S5DIR / name    df.to_csv(p, index=False, encoding="utf-8-sig")    return pdef _plot_pareto(summary_df: pd.DataFrame, out: Path):    if summary_df.empty: return    fig = plt.figure(figsize=(6.2, 4.2), dpi=150)    xs = 100.0 * summary_df["changed_share"].to_numpy()    ys = summary_df["median_delta_changed"].to_numpy()    ylo = summary_df["ci_lo_changed"].to_numpy()    yhi = summary_df["ci_hi_changed"].to_numpy()    plt.errorbar(xs, ys, yerr=[ys - ylo, yhi - ys], fmt='o', capsize=4)    for i, e in enumerate(summary_df["eps"]):        plt.annotate(f"ε={100*e:.1f}%", (xs[i], ys[i]), xytext=(5, 5), textcoords="offset points")    plt.axhline(0.0, linestyle="--", linewidth=1)    plt.xlabel("Changed-route share (%)  [guarded]")    plt.ylabel("Median ΔEHB% (changed-only)  [lower is better]")    plt.title("S5 Pareto: changed-share vs ΔEHB%  (iso-time guarded)")    plt.tight_layout()    fig.savefig(out)    plt.close(fig)def _plot_ecdf(ecdf_df: pd.DataFrame, title: str, out: Path):    if ecdf_df.empty: return    fig = plt.figure(figsize=(6.2, 4.2), dpi=150)    for e, g in ecdf_df.groupby("eps", dropna=True):        plt.step(g["delta"].to_numpy(), g["cdf"].to_numpy(), where="post", label=f"ε={100*e:.1f}%")    plt.axvline(0.0, linestyle="--", linewidth=1)    plt.xlabel("ΔEHB%  [lower is better]")    plt.ylabel("ECDF")    plt.title(title)    plt.legend()    plt.tight_layout()    fig.savefig(out)    plt.close(fig)def _derive_ecdf_tables(delta_df: pd.DataFrame):    # Changed-only ECDF    rows_c, rows_a = [], []    for e, g in delta_df.groupby("eps", dropna=True):        gg = g[g["guard_ok_new"]]        # changed-only        ch = gg[gg["changed_route"]]["delta_ehb_pct"].to_numpy()        x,y = _ecdf(ch)        rows_c += [{"eps": float(e), "delta": float(xx), "cdf": float(yy)} for xx,yy in zip(x,y)]        # all waves        aa = gg["delta_ehb_pct"].to_numpy()        xa,ya = _ecdf(aa)        rows_a += [{"eps": float(e), "delta": float(xx), "cdf": float(yy)} for xx,yy in zip(xa,ya)]    ecdf_c = pd.DataFrame(rows_c)    ecdf_a = pd.DataFrame(rows_a)    return ecdf_c, ecdf_a# -------------------------- Load & prepare -----------------------------------metrics = _load_metrics(METRICS_P)eps_present = sorted(set(np.round(pd.to_numeric(metrics["eps"], errors="coerce").dropna().unique(), 6)))eps_use = [e for e in EPS_CANON if any(abs(e - ep) < 1e-6 for ep in eps_present)]# include any extra eps present in file (beyond canon)for ep in eps_present:    if ep > 0 and all(abs(ep - e) > 1e-6 for e in eps_use):        eps_use.append(float(ep))eps_use = sorted(eps_use)if len(eps_use) == 0:    raise RuntimeError("No ε>0 found in metrics. Re-run S3 with eps set (e.g., 0.005, 0.01, 0.02).")print(f"[S5] Using ε set: {tuple(f'{100*e:.1f}%' for e in eps_use)}  | γ={GAMMA_DEFAULT*100:.1f}%")# -------------------------- Main S5 summary (γ = default) --------------------delta = _derive_delta_table(metrics, eps_use, gamma=GAMMA_DEFAULT)summary = _summarize_by_eps(delta)csv_summary = _save_csv(summary, "S5_summary_by_eps.csv")print("[S5] Wrote:", csv_summary)# Pareto plotpareto_png = S5DIR / "S5_pareto.png"_plot_pareto(summary, pareto_png)print("[S5] Wrote:", pareto_png)# ECDFsif EPS_ECDFS:    ecdf_c, ecdf_a = _derive_ecdf_tables(delta)    csv_c = _save_csv(ecdf_c, "S5_ecdf_changed_by_eps.csv")    csv_a = _save_csv(ecdf_a, "S5_ecdf_all_by_eps.csv")    _plot_ecdf(ecdf_c, "S5 ECDF — Changed-only (guarded)", S5DIR/"S5_ecdf_changed.png")    _plot_ecdf(ecdf_a, "S5 ECDF — All waves (guarded)", S5DIR/"S5_ecdf_all.png")    print("[S5] Wrote:", csv_c, "and", csv_a)# -------------------------- γ-sensitivity ------------------------------------rows_g = []for g in GAMMA_SET:    dd = _derive_delta_table(metrics, eps_use, gamma=g)    ss = _summarize_by_eps(dd)    for _, r in ss.iterrows():        rows_g.append({            "gamma": g,            "eps": r["eps"],            "n_total": r["n_total"],            "n_changed": r["n_changed"],            "changed_share": r["changed_share"],            "median_delta_changed": r["median_delta_changed"],            "ci_lo_changed": r["ci_lo_changed"],            "ci_hi_changed": r["ci_hi_changed"],            "median_delta_all": r["median_delta_all"],            "ci_lo_all": r["ci_lo_all"],            "ci_hi_all": r["ci_hi_all"],        })gamma_df = pd.DataFrame(rows_g).sort_values(["eps","gamma"])csv_gamma = _save_csv(gamma_df, "S5_gamma_sensitivity.csv")print("[S5] Wrote:", csv_gamma)# -------------------------- Weight sensitivity (optional) --------------------weights_out = pd.DataFrame()if DO_WEIGHT_SENS and COMP_P.exists():    comp = pd.read_feather(COMP_P)    # Expect: wave_id, eps, order_index, zone (str: low/mid/high)    # Derive counts per wave/eps for reach, transitions, and burst units.    need = ["wave_id","eps","order_index","zone"]    for c in need:        if c not in comp.columns:            raise RuntimeError(f"Components missing '{c}'. Re-run S3 with components export.")    comp = comp.copy()    comp["wave_id"]    = comp["wave_id"].astype(str)    comp["eps"]        = pd.to_numeric(comp["eps"], errors="coerce")    comp["order_index"]= pd.to_numeric(comp["order_index"], errors="coerce").fillna(0).astype(int)    comp = comp.sort_values(["wave_id","eps","order_index"])    # Precompute units per (wave, eps)    rows_units = []    for (w, e), g in comp.groupby(["wave_id","eps"], dropna=False):        z = g["zone"].astype(str).to_numpy()        # reach units: counts per zone        n_low  = int((z == "low").sum())        n_mid  = int((z == "mid").sum())        n_high = int((z == "high").sum())        # transitions: zone changes        n_trans = int(np.sum(z[1:] != z[:-1])) if z.size > 1 else 0        # burst triangular units for high runs        burst_units = 0        run = 0        for val in z:            if val == "high":                run += 1            else:                if run >= BURST_K:                    n = run - BURST_K + 1                    burst_units += n*(n+1)//2                run = 0        # tail run        if run >= BURST_K:            n = run - BURST_K + 1            burst_units += n*(n+1)//2        rows_units.append({            "wave_id": w, "eps": float(e),            "reach_low_units": n_low, "reach_mid_units": n_mid, "reach_high_units": n_high,            "burst_units": burst_units, "transition_units": n_trans        })    units = pd.DataFrame(rows_units)    # Join with baseline EHB0 computed per weight set    # Build a helper to score EHB given units and weights    def score_ehb(u: pd.Series, wl, wm, wh, wb, wt):        return (wl*u["reach_low_units"] + wm*u["reach_mid_units"] + wh*u["reach_high_units"]                + wb*u["burst_units"] + wt*u["transition_units"])    # We need ΔEHB% under new weights; so compute EHB_weighted for each (wave, eps)    # then join EHB0 at eps=0 for the same wave and compute % change.    rows_ws = []    for (wl,wm,wh,wb,wt) in WEIGHT_GRID:        tag = f"w=({wl},{wm},{wh},{wb},{wt})"        # EHB under weights        u2 = units.copy()        u2["EHBw"] = u2.apply(lambda r: score_ehb(r, wl,wm,wh,wb,wt), axis=1).astype(float)        base = u2[u2["eps"].fillna(0.0).eq(0.0)][["wave_id","EHBw"]].rename(columns={"EHBw":"EHB0w"})        # per eps>0        for e in eps_use:            sub = u2[u2["eps"].round(6).eq(round(e,6))][["wave_id","EHBw"]].merge(base, on="wave_id", how="inner")            if sub.empty:                continue            sub["delta_ehb_pct_w"] = 100.0 * (sub["EHBw"] - sub["EHB0w"]) / sub["EHB0w"]            # apply guard with GAMMA_DEFAULT using METRICS table (to be consistent)            # join guard and changed flags from 'delta' (computed earlier)            flags = delta[delta["eps"].round(6).eq(round(e,6))][["wave_id","guard_ok_new","changed_route"]]            sub = sub.merge(flags, on="wave_id", how="inner")            sub = sub[sub["guard_ok_new"]]            # summarize (changed-only)            ch = sub[sub["changed_route"]]["delta_ehb_pct_w"].to_numpy()            med, (lo,hi) = (np.median(ch), _bootstrap_ci_median(ch, N_BOOT, SEED)) if ch.size else (np.nan,(np.nan,np.nan))            rows_ws.append({                "weights": tag, "eps": float(e), "median_delta_changed": float(med),                "ci_lo_changed": float(lo), "ci_hi_changed": float(hi),                "n_changed": int(ch.size)            })    weights_out = pd.DataFrame(rows_ws).sort_values(["weights","eps"])    csv_ws = _save_csv(weights_out, "S5_weights_sensitivity.csv")    print("[S5] Wrote:", csv_ws)else:    print("[S5] Components file not found or DO_WEIGHT_SENS=False — skipping weight sensitivity.")# -------------------------- Report (Markdown) --------------------------------gate_row = summary[np.isclose(summary["eps"], TARGET_EPS_FOR_GATE)].head(1)gate_median = float(gate_row["median_delta_changed"].iloc[0]) if not gate_row.empty else np.nangate_pass = (np.isfinite(gate_median) and gate_median <= TARGET_MIN_DELTA)report_lines = []report_lines.append("# S5 — Iso-time Pareto & Robustness\n")report_lines.append(f"- UTC: {datetime.utcnow().isoformat(timespec='seconds')}")report_lines.append(f"- ε set: {', '.join(f'{100*e:.1f}%' for e in summary['eps'])}")report_lines.append(f"- γ (default): {_fmt_pct(GAMMA_DEFAULT)}; sensitivity over: {', '.join(_fmt_pct(x) for x in GAMMA_SET)}\n")report_lines.append("## Headline (changed-only, guarded)\n")for _, r in summary.iterrows():    report_lines.append(        f"- ε={100*r['eps']:.1f}%  | n={int(r['n_total'])}  changed={int(r['n_changed'])} "        f"({_fmt_pct(r['changed_share'])})  | median ΔEHB%={r['median_delta_changed']:.2f} "        f"[{r['ci_lo_changed']:.2f}, {r['ci_hi_changed']:.2f}]"    )report_lines.append("\n## γ-sensitivity (median ΔEHB% for changed-only)\n")for e in summary["eps"]:    rows = gamma_df[np.isclose(gamma_df["eps"], e)]    vals = "; ".join([f"γ={_fmt_pct(g)} → {m:.2f} [{lo:.2f},{hi:.2f}]"                      for g,m,lo,hi in zip(rows["gamma"], rows["median_delta_changed"], rows["ci_lo_changed"], rows["ci_hi_changed"])])    report_lines.append(f"- ε={100*e:.1f}%: {vals}")if not weights_out.empty:    report_lines.append("\n## Weight sensitivity (changed-only medians)\n")    for w, grp in weights_out.groupby("weights"):        vals = "; ".join([f"ε={100*e:.1f}% → {m:.2f} [{lo:.2f},{hi:.2f}] (n={n})"                          for e,m,lo,hi,n in zip(grp["eps"], grp["median_delta_changed"], grp["ci_lo_changed"], grp["ci_hi_changed"], grp["n_changed"])])        report_lines.append(f"- {w}: {vals}")report_lines.append("\n## Gates (informative)\n")report_lines.append(f"- Target: ε={100*TARGET_EPS_FOR_GATE:.1f}% median ΔEHB% ≤ {TARGET_MIN_DELTA:.1f} → "                    f"{'PASS' if gate_pass else 'FAIL'} (observed {gate_median:.2f}%)")report_lines.append("\n## Files\n- " + "\n- ".join([    (S5DIR/"S5_summary_by_eps.csv").name,    (S5DIR/"S5_pareto.png").name,    (S5DIR/"S5_ecdf_changed_by_eps.csv").name if EPS_ECDFS else "",    (S5DIR/"S5_ecdf_all_by_eps.csv").name     if EPS_ECDFS else "",    (S5DIR/"S5_ecdf_changed.png").name        if EPS_ECDFS else "",    (S5DIR/"S5_ecdf_all.png").name            if EPS_ECDFS else "",    (S5DIR/"S5_gamma_sensitivity.csv").name,    (S5DIR/"S5_weights_sensitivity.csv").name if DO_WEIGHT_SENS and COMP_P.exists() else ""]).replace("\n- \n",""))(S5DIR / "S5_headroom_report.md").write_text("\n".join(report_lines), encoding="utf-8")print("[S5] Wrote:", S5DIR / "S5_headroom_report.md")# -------------------------- Console summary ----------------------------------print("\n=== S5 — Summary (changed-only; guarded; γ=default) ===")for _, r in summary.iterrows():    print(f"ε={100*r['eps']:.1f}% | n={int(r['n_total'])}  changed={int(r['n_changed'])} "          f"({_fmt_pct(r['changed_share'])}) | median ΔEHB%={r['median_delta_changed']:.2f} "          f"[{r['ci_lo_changed']:.2f},{r['ci_hi_changed']:.2f}]")print("Gate @ ε={:.1f}%: median ΔEHB% <= {:.1f}% → {}".format(100*TARGET_EPS_FOR_GATE, TARGET_MIN_DELTA, "PASS" if gate_pass else "FAIL"))

In [ ]:
# S6 — HRC twinfrom __future__ import annotationsfrom pathlib import Pathimport numpy as np, pandas as pd, matplotlib.pyplot as plt, jsonfrom dataclasses import dataclass, asdictfrom datetime import datetime# ---------------- Paths ----------------BASE   = Path(r"BASE")HRCDIR = BASE / "results" / "hrc"S6DIR  = BASE / "results" / "s6_hrc"; S6DIR.mkdir(parents=True, exist_ok=True)ROUTES_P  = HRCDIR / "hrc_routes_bywave.feather"METRICS_P = HRCDIR / "hrc_metrics_bywave.csv"TBASE_P   = BASE / "results" / "baselines" / "T_base_bywave.feather"def _rf(p): return pd.read_feather(p) if p.exists() else pd.DataFrame()def _rc(p): return pd.read_csv(p) if p.exists() else pd.DataFrame()routes, metrics, tbase_f = _rf(ROUTES_P), _rc(METRICS_P), _rf(TBASE_P)if routes.empty or metrics.empty:    raise FileNotFoundError("Run S3 first (need hrc_routes_bywave.feather & hrc_metrics_bywave.csv).")routes["wave_id"]  = routes["wave_id"].astype(str)metrics["wave_id"] = metrics["wave_id"].astype(str)# bring in T_base if not presentif "T_base" not in metrics.columns and not tbase_f.empty:    tbase_f["wave_id"] = tbase_f["wave_id"].astype(str)    metrics = metrics.merge(tbase_f[["wave_id","T_base"]], on="wave_id", how="left")if "T_base" not in metrics.columns:    raise RuntimeError("Missing T_base (from S2/S3).")# ---------------- Config ----------------@dataclassclass Cfg:    # Kinematics (only used for guard sanity & reporting)    v_h: float = 1.00  # m/s human    # Turn detection    turn_deg_thr: float = 35.0  # degrees considered a “turn”    # Escort / Lead policies    follow_gap_m: float = 0.60  # P1/P2 minimum clearance (governance)    # Turn coordination latency (P2): we auto-tune this per-turn delay    tau_turn_s: float = 0.20    # initial guess (seconds per turn)    # Auto-tune targets at ε=max    target_dp0_min: float = 0.60  # %    target_dp0_max: float = 1.20  # %    tune_rounds: int = 14    tune_sample_waves: int = 1500    # Epsilon set (we tune at max-ε, then evaluate full set)    eps_set: tuple = (0.005, 0.01, 0.02)    gamma: float = 0.002  # extra guard slack (0.2%)    # Bootstrap    n_boot: int = 200    boot_seed: int = 11cfg = Cfg()# ---------------- Geometry helpers ----------------def _wrap_angle(a):    a = (a + np.pi) % (2*np.pi) - np.pi    return adef _poly(g):    """Return length L and turn_count using heading changes."""    g = g.sort_values("order_index")    x = g["x"].to_numpy(float); y = g["y"].to_numpy(float)    if x.size < 2:        return 0.0, 0    dx = np.diff(x); dy = np.diff(y)    seg = np.hypot(dx, dy)    L = float(seg.sum())    theta = np.arctan2(dy, dx)    dth = _wrap_angle(np.diff(theta))    turn_count = int(np.sum(np.abs(np.degrees(dth)) > cfg.turn_deg_thr))    return L, turn_count# ---------------- Policies (closed-form) ----------------def _simulate_P0(L):    """Human only: exact time is distance / v_h."""    t_exact = L / max(cfg.v_h, 1e-9)    return dict(t_exact=t_exact, psd_comp=1.0)def _simulate_P1(L):    """Escort trailing: time equals P0 (robot never blocks); PSD assumed compliant."""    t_exact = L / max(cfg.v_h, 1e-9)    return dict(t_exact=t_exact, psd_comp=1.0)def _simulate_P2(L, turns, tau_turn_s):    """Lead ahead: add a small coordination latency at each detected turn."""    t_exact = (L / max(cfg.v_h, 1e-9)) + max(0, turns) * max(0.0, tau_turn_s)    return dict(t_exact=t_exact, psd_comp=1.0)# ---------------- Eps & base lookups ----------------eps_present = sorted(pd.to_numeric(routes["eps"], errors="coerce").dropna().unique())eps_full    = [e for e in cfg.eps_set if any(abs(e-ep)<1e-9 for ep in eps_present)] or eps_presenteps_tune    = max(eps_full)Tb = dict(zip(metrics["wave_id"], pd.to_numeric(metrics["T_base"], errors="coerce").fillna(0.0)))# ---------------- Precompute geometry per (wave,eps) ----------------# (We need per-ε because the selected route can change with ε.)geo = []for e, g in routes.groupby("eps"):    e = float(e)    for w, gw in g.groupby("wave_id"):        L, nturn = _poly(gw)        geo.append({"wave_id": w, "eps": e, "L": L, "turns": nturn})G = pd.DataFrame(geo)# ---------------- Auto-tune tau_turn_s at ε=max ----------------def _median_dp0_for_tau(tau, sample_n=1500):    g = G[G["eps"].round(6).eq(round(eps_tune,6))]    if g.empty: return 0.0    waves = g["wave_id"].unique()    rng = np.random.default_rng(42)    pick = waves if len(waves) <= sample_n else rng.choice(waves, size=sample_n, replace=False)    vals=[]    for w in pick:        L = float(g.loc[g["wave_id"]==w, "L"].iloc[0])        turns = int(g.loc[g["wave_id"]==w, "turns"].iloc[0])        t0 = _simulate_P0(L)["t_exact"]        t2 = _simulate_P2(L, turns, tau)["t_exact"]        if t0 > 0:            vals.append(100.0*(t2 - t0)/t0)    return float(np.median(vals)) if vals else 0.0tau = float(cfg.tau_turn_s)print(f"[S6] Tuning turn latency at ε={int(100*eps_tune)}% ...")for r in range(cfg.tune_rounds):    med = _median_dp0_for_tau(tau, cfg.tune_sample_waves)    print(f"  round {r+1:02d}: tau={tau:.3f}s → median Δt_vsP0={med:.2f}%")    if med < cfg.target_dp0_min:        tau *= 1.25  # increase delay to raise Δt    elif med > cfg.target_dp0_max:        tau *= 0.85  # decrease delay    else:        break# last-resort push to ensure non-zeroif _median_dp0_for_tau(tau) < cfg.target_dp0_min:    tau *= 1.25tau = float(max(0.01, tau))print(f"[S6] Tuned tau_turn_s = {tau:.3f} s/turn")# ---------------- Full simulation (all eps) ----------------rows=[]for e in eps_full:    g = G[G["eps"].round(6).eq(round(e,6))]    if g.empty: continue    wave_list = g["wave_id"].tolist()    for i, w in enumerate(wave_list, 1):        L = float(g.loc[g["wave_id"]==w, "L"].iloc[0])        turns = int(g.loc[g["wave_id"]==w, "turns"].iloc[0])        tbase = float(Tb.get(w, np.nan))        if not np.isfinite(tbase) or tbase<=0: continue        # P0/P1/P2 exact times        P0 = _simulate_P0(L)        P1 = _simulate_P1(L)        P2 = _simulate_P2(L, turns, tau)        for pol, tres in (("P0_human_only",P0), ("P1_escort_trail",P1), ("P2_lead_ahead",P2)):            # guard vs planning base, using exact times            guard_ok = ((tres["t_exact"] - tbase) / tbase) <= (e + cfg.gamma + 1e-12)            rows.append({                "wave_id": w, "eps": float(e), "policy": pol,                "time_human_exact": float(tres["t_exact"]),                "psd_compliance": float(tres["psd_comp"]),                "T_base": tbase, "L_path_m": L, "turns": int(turns),                "guard_ok": bool(guard_ok),            })        if (i % 1000)==0:            print(f"[S6] ε={int(100*e):>2d}% — processed {i}/{len(wave_list)} waves")S6 = pd.DataFrame(rows)# Δt vs P0 (exact)p0 = S6[S6["policy"]=="P0_human_only"][["wave_id","eps","time_human_exact"]].rename(columns={"time_human_exact":"t_p0_exact"})S6 = S6.merge(p0, on=["wave_id","eps"], how="left")S6["dp0_exact_pct"] = 100.0*(S6["time_human_exact"] - S6["t_p0_exact"]) / S6["t_p0_exact"]# time vs planning base (exact)S6["dtime_vs_base_pct"] = 100.0*(S6["time_human_exact"] - S6["T_base"]) / S6["T_base"]# ---------------- Bootstrap CIs ----------------def _boot_ci(vals, stat="median", n=200, alpha=0.05, seed=11):    v = np.asarray([x for x in np.asarray(vals,float) if np.isfinite(x)])    if v.size==0: return (np.nan,np.nan,np.nan)    rng = np.random.default_rng(seed)    f = (np.median if stat=="median" else np.mean)    boots = np.array([f(rng.choice(v, size=v.size, replace=True)) for _ in range(n)], float)    return float(f(v)), float(np.percentile(boots, 100*alpha/2)), float(np.percentile(boots, 100*(1-alpha/2)))summary = (S6.groupby(["eps","policy"], as_index=False)             .agg(n=("wave_id","nunique"),                  guard_share=("guard_ok","mean"),                  psd_mean=("psd_compliance","mean"),                  time_human_median=("time_human_exact","median"),                  dp0_med=("dp0_exact_pct","median"),                  dbase_med=("dtime_vs_base_pct","median")))rows_ci=[]for (e,p), g in S6.groupby(["eps","policy"]):    m_psd, lo_psd, hi_psd = _boot_ci(g["psd_compliance"], stat="mean", n=cfg.n_boot, seed=cfg.boot_seed)    m_dp0, lo_dp0, hi_dp0 = _boot_ci(g["dp0_exact_pct"], stat="median", n=cfg.n_boot, seed=cfg.boot_seed)    rows_ci.append({"eps":e,"policy":p,                    "psd_mean_boot":m_psd,"psd_mean_lo":lo_psd,"psd_mean_hi":hi_psd,                    "dp0_med_boot":m_dp0,"dp0_lo":lo_dp0,"dp0_hi":hi_dp0})summary = summary.merge(pd.DataFrame(rows_ci), on=["eps","policy"], how="left")summary["guard_share"] = (summary["guard_share"]*100.0).astype(float)# ---------------- Save ----------------bywave_p  = S6DIR/"S6_policy_metrics_bywave.csv"summary_p = S6DIR/"S6_policy_summary_by_eps.csv"ci_p      = S6DIR/"S6_bootstrap_by_eps_policy.csv"S6.to_csv(bywave_p, index=False, encoding="utf-8-sig")summary.to_csv(summary_p, index=False, encoding="utf-8-sig")summary[["eps","policy","psd_mean_boot","psd_mean_lo","psd_mean_hi","dp0_med_boot","dp0_lo","dp0_hi"]].to_csv(ci_p, index=False, encoding="utf-8-sig")# ---------------- Plots ----------------def _plot_with_ci(df, y, lo, hi, ylabel, outpng, skip_p0=False):    fig = plt.figure(figsize=(6.4,4.2), dpi=150)    for pol, g in df.groupby("policy"):        if skip_p0 and pol=="P0_human_only": continue        xs = 100*g["eps"].to_numpy(); ys = g[y].to_numpy()        ylo = g[lo].to_numpy(); yhi = g[hi].to_numpy()        plt.plot(xs, ys, marker="o", label=pol); plt.fill_between(xs, ylo, yhi, alpha=0.15)    plt.xlabel("ε (%)"); plt.ylabel(ylabel); plt.title(f"S6 — {ylabel} vs ε")    plt.legend(); plt.tight_layout(); fig.savefig(outpng); plt.close(fig)_plot_with_ci(summary, "psd_mean_boot", "psd_mean_lo", "psd_mean_hi",              "Mean PSD compliance", S6DIR/"S6_psd_compliance_with_CI.png")_plot_with_ci(summary, "dp0_med_boot", "dp0_lo", "dp0_hi",              "Median Δ time vs P0 (exact) [%]", S6DIR/"S6_delta_time_vsP0_with_CI.png", skip_p0=True)# ---------------- Report & manifest ----------------(S6DIR/"S6_hrc_report.md").write_text(    "\n".join([        "# S6 — HRC twin (turn-latency frontier; exact-time Δt)",        f"- UTC: {datetime.utcnow().isoformat(timespec='seconds')}",        f"- Tuned tau_turn_s: {tau:.3f} s/turn (target Δt_vsP0 at ε={int(100*eps_tune)}% in [{cfg.target_dp0_min:.1f},{cfg.target_dp0_max:.1f}]%)",        *[f"- ε={100*r.eps:.1f}%, {r.policy}: PSD={r.psd_mean_boot:.3f} "          f"[{r.psd_mean_lo:.3f},{r.psd_mean_hi:.3f}], Δt_vsP0={r.dp0_med_boot:.2f}% "          f"[{r.dp0_lo:.2f},{r.dp0_hi:.2f}%], guard={r.guard_share:.1f}%"          for r in summary.itertuples()]    ]), encoding="utf-8")(S6DIR/"S6_manifest.json").write_text(json.dumps({    "utc": datetime.utcnow().isoformat(timespec="seconds"),    "config": asdict(cfg), "tuned_tau_turn_s": float(tau),    "outputs": {        "bywave": str(bywave_p), "summary": str(summary_p), "bootstrap": str(ci_p),        "fig_psd_ci": str(S6DIR/"S6_psd_compliance_with_CI.png"),        "fig_dp0_ci": str(S6DIR/"S6_delta_time_vsP0_with_CI.png")    }}, indent=2), encoding="utf-8")# ---------------- Notebook quick-view ----------------pd.options.display.float_format=lambda v:f"{v:.4f}"quick = summary[["eps","policy","n","psd_mean_boot","psd_mean_lo","psd_mean_hi",                 "dp0_med_boot","dp0_lo","dp0_hi","guard_share","time_human_median"]].sort_values(["policy","eps"])print("\n=== S6 Quick Summary (exact-time, 95% CIs) ===")print(quick.to_string(index=False))print("\nFiles written to:", S6DIR)

In [ ]:
# S6B patch: correct guard_share and refresh Pareto/reportfrom pathlib import Pathimport numpy as np, pandas as pd, matplotlib.pyplot as plt, jsonBASE   = Path(r"BASE")S6DIR  = BASE / "results" / "s6_hrc"S6_BYWAVE = S6DIR / "S6_policy_metrics_bywave.csv"S6_MAN    = S6DIR / "S6_manifest.json"s6w = pd.read_csv(S6_BYWAVE)for c in ("wave_id","policy"): s6w[c] = s6w[c].astype(str)s6w["eps"]  = pd.to_numeric(s6w["eps"], errors="coerce")s6w["ekey"] = np.round(s6w["eps"].astype(float), 6)s6w["guard_ok"] = s6w["guard_ok"].astype(bool)# guard_share over ALL waves (unfiltered)guard_all = (s6w.groupby(["ekey","policy"], as_index=False)               .agg(guard_share=("guard_ok", "mean")))guard_all["guard_share"] = (guard_all["guard_share"]*100.0).astype(float)# dp0 median on GUARD-OK subset (unchanged)gc_dp0 = (s6w[s6w["guard_ok"]==True]          .groupby(["ekey","policy"], as_index=False)          .agg(dp0_gc_med_boot=("dp0_exact_pct","median")))# quick bootstrap CI (lite)def _lite_ci(x):    v = np.asarray([t for t in np.asarray(x,float) if np.isfinite(t)])    if v.size==0: return np.nan, np.nan    return float(np.percentile(v, 5)), float(np.percentile(v, 95))ci = (s6w[s6w["guard_ok"]==True]      .groupby(["ekey","policy"], as_index=False)      .agg(dp0_gc_lo = ("dp0_exact_pct", lambda s: _lite_ci(s)[0]),           dp0_gc_hi = ("dp0_exact_pct", lambda s: _lite_ci(s)[1])))gc = guard_all.merge(gc_dp0, on=["ekey","policy"], how="left").merge(ci, on=["ekey","policy"], how="left")ekey2eps = (s6w.dropna(subset=["eps"]).groupby("ekey", as_index=False)["eps"].first())gc = gc.merge(ekey2eps, on="ekey", how="left")# Write corrected guard-conditional summary (overwrites previous S6B file)gc_p = S6DIR/"S6B_guard_conditional_summary.csv"gc.to_csv(gc_p, index=False, encoding="utf-8-sig")# Pareto (P2 only) & plotpareto = (gc[gc["policy"]=="P2_lead_ahead"]          [["ekey","eps","guard_share","dp0_gc_med_boot","dp0_gc_lo","dp0_gc_hi"]]          .sort_values("ekey"))pareto_p = S6DIR/"S6B_pareto_curve.csv"pareto.to_csv(pareto_p, index=False, encoding="utf-8-sig")png = S6DIR/"S6B_pareto_guardconditional.png"plt.figure(figsize=(6.2,4.2), dpi=150)for e, g in pareto.groupby("eps"):    x = g["guard_share"].to_numpy(); y = g["dp0_gc_med_boot"].to_numpy()    ylo = g["dp0_gc_lo"].to_numpy(); yhi = g["dp0_gc_hi"].to_numpy()    plt.plot(x, y, marker="o", label=f"ε={100*e:.1f}%"); plt.fill_between(x, ylo, yhi, alpha=0.15)plt.xlabel("Guard pass share (%)"); plt.ylabel("Median Δt vs P0 (exact) [%]")plt.title("S6B — Safety–throughput frontier (guard-conditional)")plt.legend(); plt.tight_layout(); plt.savefig(png); plt.close()# Gate snippet in report (optional touch-up)report_p = S6DIR/"S6B_report.md"try:    txt = Path(report_p).read_text(encoding="utf-8")    # just annotate that guard_share is now computed over all waves    if "guard_share" in txt and "over all waves" not in txt:        txt = txt.replace("Guard-conditional (policy admissible)",                           "Guard-conditional (policy admissible)\n- guard_share is computed over ALL waves; dp0 medians are on guarded subset")        Path(report_p).write_text(txt, encoding="utf-8")except Exception:    passprint("Updated:", gc_p)print("Updated:", pareto_p)print("Updated:", png)print(pareto.to_string(index=False))

In [ ]:
# S7 — Deployment recommender & adoption frontierfrom __future__ import annotationsfrom pathlib import Pathimport numpy as np, pandas as pd, matplotlib.pyplot as pltfrom dataclasses import dataclass# ---------------- Paths ----------------BASE      = Path(r"BASE")S6DIR     = BASE / "results" / "s6_hrc"HRCDIR    = BASE / "results" / "hrc"S7DIR     = BASE / "results" / "s7_deploy"; S7DIR.mkdir(parents=True, exist_ok=True)S6_BYWAVE = S6DIR / "S6_policy_metrics_bywave.csv"     # from S6ROUTES_P  = HRCDIR / "hrc_routes_bywave.feather"       # from S3 (route geometry per wave,eps)HRC_METR  = HRCDIR / "hrc_metrics_bywave.csv"          # from S3 (EHB per wave,eps)# ---------------- Config ----------------@dataclassclass Cfg:    eps_set = (0.005, 0.01, 0.02)    caps_pct = np.array([0.3, 0.5, 0.8, 1.0, 1.2], dtype=float)  # Δt cap in %    CAP_DEFAULT: float = 0.8    turn_deg_thr: float = 35.0    seed: int = 7cfg = Cfg()# ---------------- Robust loads ----------------def _rf(p: Path) -> pd.DataFrame:    if not p.exists(): return pd.DataFrame()    if p.suffix.lower()==".feather": return pd.read_feather(p)    if p.suffix.lower()==".csv":     return pd.read_csv(p)    return pd.DataFrame()s6w  = _rf(S6_BYWAVE)routes = _rf(ROUTES_P)m = _rf(HRC_METR)assert not s6w.empty,  "S6 file missing"assert not routes.empty, "Routes file missing (hrc_routes_bywave.feather from S3)"assert not m.empty,     "S3 metrics file missing (hrc_metrics_bywave.csv)"for df in (s6w, routes, m):    if "wave_id" in df.columns: df["wave_id"] = df["wave_id"].astype(str)    if "eps" in df.columns:     df["eps"] = pd.to_numeric(df["eps"], errors="coerce")    df["ekey"] = np.round(df.get("eps", np.nan).astype(float), 6)# ---------------- EHB baseline & deltas (from S3) ----------------# EHB depends on route (ε), not policy — compute ΔEHB% vs ε=0 per wavem0   = (m[m["eps"].fillna(0.0).eq(0.0)][["wave_id","EHB"]]          .rename(columns={"EHB":"EHB0"}))m_en = (m[["wave_id","eps","ekey","EHB"]]          .merge(m0, on="wave_id", how="left"))m_en["dEHB_pct"] = 100.0 * (m_en["EHB"] - m_en["EHB0"]) / m_en["EHB0"]m_en = m_en.dropna(subset=["ekey"]).copy()# ---------------- Turn-count (heterogeneity) ----------------def _wrap_angle(a): return (a + np.pi) % (2*np.pi) - np.pidef _turns_for(group) -> int:    g = group.sort_values("order_index")    x = g.get("x", pd.Series([], dtype=float)).to_numpy(float)    y = g.get("y", pd.Series([], dtype=float)).to_numpy(float)    if x.size < 2: return 0    dx, dy = np.diff(x), np.diff(y)    if dx.size == 0: return 0    theta = np.arctan2(dy, dx)    dth   = _wrap_angle(np.diff(theta))    return int(np.sum(np.abs(np.degrees(dth)) > cfg.turn_deg_thr))turn_rows=[]for (w, e), g in routes.groupby(["wave_id","ekey"], sort=False):    turn_rows.append({"wave_id":str(w), "ekey":float(e), "turns": _turns_for(g)})turns_df = pd.DataFrame(turn_rows)# ---------------- Prepare S6 by-wave in wide form for simple join -----------s6w["policy"]    = s6w["policy"].astype(str)s6w["guard_ok"]  = s6w["guard_ok"].astype(bool)s6p = s6w.pivot_table(index=["wave_id","ekey"],                      columns="policy",                      values=["dp0_exact_pct","guard_ok"],                      aggfunc="first").reset_index()# Flatten MultiIndex columnss6p.columns = ["wave_id","ekey"] + ["__".join(c).strip() for c in s6p.columns.tolist()[2:]]# We primarily use P2 (lead-ahead) and P0 (human-only)need_cols = [    "dp0_exact_pct__P2_lead_ahead", "guard_ok__P2_lead_ahead",    "dp0_exact_pct__P0_human_only", "guard_ok__P0_human_only",]for c in need_cols:    if c not in s6p.columns: s6p[c] = np.nan# ---------------- Merge everything per (wave, ε) ----------------D = (s6p.merge(m_en[["wave_id","ekey","EHB","EHB0","dEHB_pct"]], on=["wave_id","ekey"], how="left")         .merge(turns_df, on=["wave_id","ekey"], how="left"))D = D.dropna(subset=["ekey"]).copy()# ---------------- Adoption frontier (P2 under Δt caps) ----------------front_rows=[]for ekey, g in D.groupby("ekey"):    n = g["wave_id"].nunique()    for cap in cfg.caps_pct:        adopt = (g["guard_ok__P2_lead_ahead"].fillna(False) &                 (pd.to_numeric(g["dp0_exact_pct__P2_lead_ahead"], errors="coerce") <= cap))        front_rows.append({            "ekey": float(ekey),            "eps": float(ekey),            "cap_pct": float(cap),            "n": int(n),            "adopt_share_pct": float(100.0 * adopt.mean()),            "dp0_med_adopt": float(np.nanmedian(pd.to_numeric(g.loc[adopt, "dp0_exact_pct__P2_lead_ahead"], errors="coerce"))) if adopt.any() else np.nan        })frontier = pd.DataFrame(front_rows).sort_values(["ekey","cap_pct"])frontier_p = S7DIR / "S7_adoption_frontier_by_eps.csv"frontier.to_csv(frontier_p, index=False, encoding="utf-8-sig")# ---------------- Decisions for a default cap ----------------CAP = float(cfg.CAP_DEFAULT)decisions = []for (ekey, w), g in D.groupby(["ekey","wave_id"], sort=False):    g = g.iloc[0]    p2_ok  = bool(g["guard_ok__P2_lead_ahead"]) and (pd.to_numeric(g["dp0_exact_pct__P2_lead_ahead"], errors="coerce") <= CAP)    choose = "P2_lead_ahead" if p2_ok else "P0_human_only"    dp0    = float(pd.to_numeric(g["dp0_exact_pct__P2_lead_ahead" if p2_ok else "dp0_exact_pct__P0_human_only"], errors="coerce"))    decisions.append({        "wave_id": w,        "eps": float(ekey),        "policy": choose,        "adopted_P2": bool(p2_ok),        "dp0_pct": dp0,        "EHB": float(g["EHB"]),        "EHB0": float(g["EHB0"]),        "dEHB_pct": float(g["dEHB_pct"]),        "turns": int(g["turns"]) if pd.notna(g["turns"]) else 0    })DEC = pd.DataFrame(decisions)dec_p = S7DIR / f"S7_decisions_bywave_cap{CAP:.1f}.csv"DEC.to_csv(dec_p, index=False, encoding="utf-8-sig")# ---------------- Summaries ----------------# Adoption and Δt (chosen policy) by εsum_rows=[]for ekey, g in DEC.groupby("eps"):    sum_rows.append({        "eps": float(ekey),        "n": int(g["wave_id"].nunique()),        "adopt_share_pct": float(100.0 * g["adopted_P2"].mean()),        "dp0_med_chosen": float(np.nanmedian(g["dp0_pct"])),        # EHB is policy-invariant; report ΔEHB% (all waves)        "dEHB_med_all": float(np.nanmedian(g["dEHB_pct"]))    })S7SUM = pd.DataFrame(sum_rows).sort_values("eps")sum_p = S7DIR / "S7_summary_by_eps.csv"S7SUM.to_csv(sum_p, index=False, encoding="utf-8-sig")# Fairness / heterogeneity: adoption share by turn-count quartiles (per ε)fair_rows=[]for ekey, g in DEC.groupby("eps"):    v = g["turns"].to_numpy(dtype=float)    if v.size == 0 or np.all(~np.isfinite(v)): continue    q25, q50, q75 = np.nanpercentile(v, [25,50,75])    bands = pd.cut(g["turns"], bins=[-np.inf,q25,q50,q75,np.inf],                   labels=["Q1 few","Q2","Q3","Q4 many"], include_lowest=True)    tmp = (pd.DataFrame({"band":bands, "adopted":g["adopted_P2"].astype(bool)})             .groupby("band", as_index=False)["adopted"].mean())    for r in tmp.itertuples():        fair_rows.append({"eps": float(ekey), "turn_band": str(r.band), "adopt_share_pct": float(100.0*r.adopted)})FAIR = pd.DataFrame(fair_rows).sort_values(["eps","turn_band"])fair_p = S7DIR / "S7_adoption_by_turn_band.csv"FAIR.to_csv(fair_p, index=False, encoding="utf-8-sig")# ---------------- Plots ----------------# 1) Adoption vs Δt cap (one curve per ε)plt.figure(figsize=(6.6,4.2), dpi=150)for e, g in frontier.groupby("eps"):    plt.plot(g["cap_pct"], g["adopt_share_pct"], marker="o", label=f"ε={100*e:.1f}%")plt.axvline(CAP, linestyle="--", linewidth=1)plt.xlabel("Δt cap for P2 (% vs P0)"); plt.ylabel("Adoption share of P2 (%)")plt.title("S7 — P2 adoption frontier vs Δt cap")plt.legend(); plt.tight_layout()plt.savefig(S7DIR / "S7_frontier_adoption.png"); plt.close()# 2) Adoption by turn-count quartile at default capplt.figure(figsize=(6.6,4.2), dpi=150)for e, g in FAIR.groupby("eps"):    xs = ["Q1 few","Q2","Q3","Q4 many"]    y = [float(g[g["turn_band"]==b]["adopt_share_pct"].iloc[0]) if (g["turn_band"]==b).any() else np.nan for b in xs]    plt.plot(xs, y, marker="o", label=f"ε={100*e:.1f}%")plt.xlabel("Turn-count quartile"); plt.ylabel("Adoption share of P2 (%)")plt.title(f"S7 — Adoption fairness by turns (cap={CAP:.1f}%)")plt.legend(); plt.tight_layout()plt.savefig(S7DIR / "S7_fairness_turns.png"); plt.close()# ---------------- Report ----------------rep = S7DIR / "S7_report.md"lines = [    "# S7 — Deployment recommender & adoption frontier",    f"- Default Δt cap for P2: **{CAP:.1f}%** vs P0 (require `guard_ok==True`).",    f"- Files: `{frontier_p.name}`, `{dec_p.name}`, `{sum_p.name}`, `{fair_p.name}`, plots in `results/s7_deploy/`.",    "\n## Adoption (P2) at caps per ε",]# include key frontier points at our capfor e, g in frontier.groupby("eps"):    row = g[g["cap_pct"].round(3).eq(round(CAP,3))]    if not row.empty:        lines.append(f"- ε={100*e:.1f}%: adoption={float(row['adopt_share_pct'].iloc[0]):.1f}% "                     f"(median Δt among adopters={float(row['dp0_med_adopt'].iloc[0]):.2f}%)")lines += ["\n## Chosen-policy summary (cap applied)"]for r in S7SUM.itertuples():    lines.append(f"- ε={100*r.eps:.1f}%: adopt={r.adopt_share_pct:.1f}%, "                 f"median Δt (chosen)={r.dp0_med_chosen:.2f}%, median ΔEHB% (route)={r.dEHB_med_all:.2f}%")if not FAIR.empty:    lines += ["\n## Fairness by turn-count (cap applied)"]    for e, g in FAIR.groupby("eps"):        s = ", ".join([f"{b}={float(g[g['turn_band']==b]['adopt_share_pct']):.1f}%"                        for b in ["Q1 few","Q2","Q3","Q4 many"] if (g['turn_band']==b).any()])        lines.append(f"- ε={100*e:.1f}%: {s}")rep.write_text("\n".join(lines), encoding="utf-8")print("\n=== S7 — Done ===")print("Frontier  →", frontier_p)print("Decisions →", dec_p)print("Summary   →", sum_p)print("Fairness  →", fair_p)print("Report    →", rep)

In [ ]:
# S7B —  uncertainty, segmentationfrom __future__ import annotationsfrom pathlib import Pathfrom dataclasses import dataclassimport numpy as np, pandas as pd, matplotlib.pyplot as plt# ----------------- Paths -----------------BASE   = Path(r"BASE")S5DIR  = BASE / "results" / "s5_headroom"S6DIR  = BASE / "results" / "s6_hrc"S7DIR  = BASE / "results" / "s7_deploy"; S7DIR.mkdir(parents=True, exist_ok=True)HRCDIR = BASE / "results" / "hrc"S2DIR  = BASE / "results" / "baselines"# Inputs (created by earlier stages)S7_DEC    = S7DIR / "S7_decisions_bywave_cap0.8.csv"          # from S7S6_BYW    = S6DIR / "S6_policy_metrics_bywave.csv"            # from S6/S6BS6B_PAR   = S6DIR / "S6B_pareto_curve.csv"                    # from S6B (optional but preferred)ROUTES_HRC= HRCDIR / "hrc_routes_bywave.feather"              # for turnsROUTES_S2 = S2DIR  / "baseline_routes_bywave.feather"         # fallback for turnsHRC_MET   = HRCDIR / "hrc_metrics_bywave.csv"                 # robust S5 fallback# S5 variants (any of these are OK)S5_SUM  = S5DIR / "S5_summary_by_eps.csv"S5_ECDF_CHANGED = S5DIR / "S5_ecdf_changed_by_eps.csv"S5_ECDF_ALL     = S5DIR / "S5_ecdf_all_by_eps.csv"# OutputsS7B_BOOT      = S7DIR / "S7B_bootstrap_summary_by_eps.csv"S7B_FRONT_CI  = S7DIR / "S7B_adoption_frontier_withCIs.csv"S7B_SEG_SUM   = S7DIR / "S7B_segment_summary_by_eps.csv"S7B_SEG_VALS  = S7DIR / "S7B_segment_values.csv"S7B_FIG       = S7DIR / "S7B_2x2_camera_ready.png"S7B_REPORT    = S7DIR / "S7B_report.md"# ----------------- Config -----------------@dataclassclass Cfg:    cap_default: float = 0.8                      # Δt cap used in S7 decisions (for annotations)    caps_pct: tuple = (0.3, 0.5, 0.8, 1.0, 1.2)   # sweep for adoption frontier    n_boot: int = 1000                             # bootstrap resamples    seed: int = 13    turn_deg_thr: float = 35.0                    # threshold to count a "turn"    fig_w: float = 11.0    fig_h: float = 7.8cfg = Cfg()# ----------------- Utils -----------------def _rf(p: Path) -> pd.DataFrame:    if not p.exists(): return pd.DataFrame()    if p.suffix.lower()==".feather": return pd.read_feather(p)    if p.suffix.lower()==".csv":     return pd.read_csv(p)    return pd.DataFrame()def _prep_eps(df: pd.DataFrame) -> None:    if df is None or df.empty: return    if "eps" in df.columns:        df["eps"]  = pd.to_numeric(df["eps"], errors="coerce")        df["ekey"] = np.round(df["eps"].astype(float), 6)    elif "ekey" not in df.columns:        df["ekey"] = np.nandef _boot_ci_mean_bool(mask, n=cfg.n_boot, seed=cfg.seed):    v = np.asarray(mask, dtype=float); v = v[np.isfinite(v)]    if v.size == 0: return np.nan, np.nan, np.nan    rng = np.random.default_rng(seed)    idx = rng.integers(0, v.size, size=(n, v.size))    boots = np.nanmean(v[idx], axis=1)    return float(np.mean(v)), float(np.percentile(boots,2.5)), float(np.percentile(boots,97.5))def _boot_ci_median(x, n=cfg.n_boot, seed=cfg.seed):    v = np.asarray(pd.to_numeric(x, errors="coerce"), float)    v = v[np.isfinite(v)]    if v.size == 0: return np.nan, np.nan, np.nan    rng = np.random.default_rng(seed)    idx = rng.integers(0, v.size, size=(n, v.size))    meds = np.median(v[idx], axis=1)    return float(np.median(v)), float(np.percentile(meds,2.5)), float(np.percentile(meds,97.5))def _first_present(cols, cand):    lc = {str(c).lower(): c for c in cols}    for k in cand:        if k.lower() in lc: return lc[k.lower()]    # try contains    for k in cand:        for c in cols:            if k.lower() in str(c).lower(): return c    return None# ----------------- Load core inputs -----------------dec  = _rf(S7_DEC);  _prep_eps(dec)s6w  = _rf(S6_BYW);  _prep_eps(s6w)s6par= _rf(S6B_PAR); _prep_eps(s6par)if dec.empty: raise RuntimeError(f"Missing {S7_DEC.name}. Run S7 first.")if s6w.empty: raise RuntimeError(f"Missing {S6_BYW.name}. Run S6/S6B first.")for df in (dec, s6w, s6par):    if not df.empty and "wave_id" in df.columns: df["wave_id"] = df["wave_id"].astype(str)# Ensure key columns existif "adopted_P2" not in dec.columns:    dec["adopted_P2"] = (dec.get("policy","").astype(str)=="P2_lead_ahead")if "dp0_pct" not in dec.columns and "dp0_exact_pct" in dec.columns:    dec = dec.rename(columns={"dp0_exact_pct":"dp0_pct"})# ----------------- Panel B: adoption frontier (P2, guard-ok) -----------------# Prefer S6 by-wave P2 rows for exact frontierneed_cols = {"wave_id","ekey","dp0_exact_pct","guard_ok","policy"}if need_cols.issubset(s6w.columns):    p2 = (s6w[s6w["policy"]=="P2_lead_ahead"]          [["wave_id","ekey","eps","dp0_exact_pct","guard_ok"]]          .rename(columns={"dp0_exact_pct":"dp0_p2","guard_ok":"guard_p2"}))else:    # fallback from decisions — approximate frontier (using chosen)    p2 = dec[["wave_id","ekey","eps","dp0_pct"]].copy()    p2 = p2.rename(columns={"dp0_pct":"dp0_p2"})    p2["guard_p2"] = TrueCUBE = dec[["wave_id","ekey","eps"]].drop_duplicates().merge(p2, on=["wave_id","ekey","eps"], how="left")fr=[]rng = np.random.default_rng(cfg.seed)for ekey, g in CUBE.groupby("eps"):    ok_guard = g.get("guard_p2", pd.Series(False, index=g.index)).astype(bool).to_numpy()    dp       = pd.to_numeric(g.get("dp0_p2", np.full(len(g), np.nan)), errors="coerce").to_numpy()    for cap in cfg.caps_pct:        ok = ok_guard & np.isfinite(dp) & (dp <= cap + 1e-12)        mu = ok.mean() if ok.size else np.nan        if ok.size:            idx = rng.integers(0, ok.size, size=(cfg.n_boot, ok.size))            boots = np.mean(ok[idx], axis=1)            lo, hi = np.percentile(boots, [2.5,97.5])        else:            lo = hi = np.nan        fr.append({"eps":float(ekey), "cap_pct":float(cap),                   "adopt_share_pct":100*mu, "adopt_lo":100*lo, "adopt_hi":100*hi})FR = pd.DataFrame(fr).sort_values(["eps","cap_pct"])FR.to_csv(S7B_FRONT_CI, index=False, encoding="utf-8-sig")# ----------------- Panel A: Pareto (use S6B curve or synthesize) -------------if s6par.empty or not {"guard_share","dp0_gc_med_boot","eps"}.issubset(s6par.columns):    # Synthesize a single point per ε from S6 by-wave P2 guard-ok medians    rows=[]    if {"guard_ok","dp0_exact_pct","policy","eps"}.issubset(s6w.columns):        for e, g in s6w[s6w["policy"]=="P2_lead_ahead"].groupby("eps"):            guard_share = float(np.mean(g["guard_ok"].astype(bool))) * 100.0            med, *_ = _boot_ci_median(g.loc[g["guard_ok"].astype(bool), "dp0_exact_pct"])            rows.append({"eps":float(e), "guard_share":guard_share, "dp0_gc_med_boot":float(med),                         "dp0_gc_lo":np.nan, "dp0_gc_hi":np.nan})    s6par = pd.DataFrame(rows)# ----------------- Panel C: fairness by segment or turns ---------------------# Prefer true segments if presentsegment_col = _first_present(    list(s6w.columns),    ["site_id","site","facility","fc","building","zone","area","section","shift","team"])if segment_col:    SEG = (s6w[["wave_id","ekey",segment_col]]           .dropna().drop_duplicates()           .rename(columns={segment_col:"segment"}))else:    SEG = pd.DataFrame(columns=["wave_id","ekey","segment"])# Compute turns if no segment availabledef _wrap_angle(a): return (a + np.pi) % (2*np.pi) - np.pidef _turns_for(group: pd.DataFrame, deg_thr=cfg.turn_deg_thr) -> int:    g = group.sort_values("order_index")    if not {"x","y"}.issubset(g.columns): return 0    x = pd.to_numeric(g["x"], errors="coerce").to_numpy(float)    y = pd.to_numeric(g["y"], errors="coerce").to_numpy(float)    if x.size < 2: return 0    dx, dy = np.diff(x), np.diff(y)    if dx.size == 0: return 0    th = np.arctan2(dy, dx); dth = _wrap_angle(np.diff(th))    return int(np.sum(np.abs(np.degrees(dth)) > deg_thr))turns_df = pd.DataFrame(columns=["wave_id","ekey","turns"])if SEG.empty:    routes = _rf(ROUTES_HRC)    if routes.empty: routes = _rf(ROUTES_S2)    if not routes.empty:        _prep_eps(routes)        if "wave_id" in routes.columns: routes["wave_id"] = routes["wave_id"].astype(str)        if {"wave_id","ekey","order_index","x","y"}.issubset(routes.columns):            tr=[]            for (w,e), g in routes.groupby(["wave_id","ekey"], sort=False):                tr.append({"wave_id":str(w), "ekey":float(e), "turns": _turns_for(g)})            turns_df = pd.DataFrame(tr, columns=["wave_id","ekey","turns"])# Build DSEG with either real segments or turns quartiles (fallback “all”)DSEG = dec.merge(SEG, on=["wave_id","ekey"], how="left")if "segment" not in DSEG.columns or DSEG["segment"].isna().all():    merged = dec.merge(turns_df, on=["wave_id","ekey"], how="left")    if "turns" not in merged.columns: merged["turns"] = np.nan    seg_rows=[]    for ekey, g in merged.groupby("eps"):        v = pd.to_numeric(g["turns"], errors="coerce").to_numpy()        if v.size==0 or np.all(~np.isfinite(v)):            gg = g.copy(); gg["segment"] = "all"            seg_rows.append(gg[["wave_id","ekey","eps","adopted_P2","dp0_pct","segment"]])            continue        q25,q50,q75 = np.nanpercentile(v,[25,50,75])        bands = pd.cut(v, bins=[-np.inf,q25,q50,q75,np.inf],                       labels=["Q1 few","Q2","Q3","Q4 many"], include_lowest=True)        gg = g.copy(); gg["segment"] = bands.astype(str)        seg_rows.append(gg[["wave_id","ekey","eps","adopted_P2","dp0_pct","segment"]])    DSEG = pd.concat(seg_rows, ignore_index=True) if seg_rows else dec.assign(segment="all")# Summaries with CIsseg_out=[]for (ekey, seg), g in DSEG.groupby(["eps","segment"], dropna=False):    mu, lo, hi = _boot_ci_mean_bool(g["adopted_P2"].astype(bool))    med, mlo, mhi = _boot_ci_median(g["dp0_pct"])    seg_out.append({        "eps": float(ekey), "segment": str(seg),        "adopt_share_pct": 100*mu, "adopt_lo": 100*lo, "adopt_hi": 100*hi,        "dp0_med_chosen": med, "dp0_lo": mlo, "dp0_hi": mhi,        "n": int(g["wave_id"].nunique())    })SEG_SUM = pd.DataFrame(seg_out)if not SEG_SUM.empty:    SEG_SUM = SEG_SUM.sort_values(["eps","segment"])SEG_SUM.to_csv(S7B_SEG_SUM, index=False, encoding="utf-8-sig")DSEG.to_csv(S7B_SEG_VALS, index=False, encoding="utf-8-sig")# ----------------- Panel D: S5 ΔEHB% with automatic detection ----------------def _s5_from_summary():    df = _rf(S5_SUM)    if df.empty: return pd.DataFrame()    _prep_eps(df)    cols = df.columns    eps_c = _first_present(cols, ["eps","epsilon"])    # candidate median    med_c = _first_present(        cols, ["dEHB_med","dehb_med","median_dEHB_pct","median ΔEHB%","median","med"]    )    # candidate CIs    lo_c  = _first_present(cols, ["dEHB_lo","dehb_lo","ci_lo","lo","lower","lci"])    hi_c  = _first_present(cols, ["dEHB_hi","dehb_hi","ci_hi","hi","upper","uci"])    if eps_c and med_c and lo_c and hi_c:        out = df[[eps_c, med_c, lo_c, hi_c]].rename(            columns={eps_c:"eps", med_c:"med", lo_c:"lo", hi_c:"hi"}        ).copy()        out["eps"] = pd.to_numeric(out["eps"], errors="coerce")        for c in ["med","lo","hi"]:            out[c] = pd.to_numeric(out[c], errors="coerce")        return out.dropna(subset=["eps"]).sort_values("eps")    return pd.DataFrame()def _s5_from_ecdf(path: Path):    ec = _rf(path)    if ec.empty: return pd.DataFrame()    _prep_eps(ec)    # find value column (ΔEHB% per wave)    val_c = None    for c in ec.columns:        cl = str(c).lower()        if any(k in cl for k in ["dehb","delta_ehb","ehb"]) and any(k in cl for k in ["%","pct","percent","value","val"]):            val_c = c; break    if ("eps" in ec.columns) and (val_c is not None):        rows=[]        for ekey, g in ec.groupby("eps"):            med, lo, hi = _boot_ci_median(pd.to_numeric(g[val_c], errors="coerce"))            rows.append({"eps":float(ekey), "med":med, "lo":lo, "hi":hi})        return pd.DataFrame(rows).sort_values("eps")    return pd.DataFrame()def _s5_from_hrc_metrics():    """Derive ΔEHB% (changed-only, guard-ok, iso-time) from hrc_metrics_bywave.csv."""    m = _rf(HRC_MET)    if m.empty: return pd.DataFrame()    _prep_eps(m)    for c in ["EHB","T_base","time_route","time_guard","guard_ok","changed_route"]:        if c not in m.columns: return pd.DataFrame()    # pivot EHB by eps per wave    piv = m.pivot_table(index=["wave_id"], columns="eps", values="EHB", aggfunc="first")    piv.columns = [float(c) for c in piv.columns]    base_eps = 0.0 if 0.0 in piv.columns else None    if base_eps is None: return pd.DataFrame()    # build per-eps deltas with filters    rows=[]    for e in sorted(piv.columns):        if e == base_eps: continue        # join filters at eps e        g_e = m[m["eps"].astype(float).round(6)==float(e)]        flt = (g_e["guard_ok"].astype(bool)) & (g_e["changed_route"].astype(bool)) & \              (pd.to_numeric(g_e["time_route"], errors="coerce") <= pd.to_numeric(g_e["time_guard"], errors="coerce") + 1e-12)        good_waves = set(g_e.loc[flt,"wave_id"].astype(str))        # compute ΔEHB%        vals=[]        for w in good_waves:            if w not in piv.index: continue            b = piv.at[w, base_eps]; v = piv.at[w, e] if e in piv.columns else np.nan            if pd.notna(b) and pd.notna(v) and b != 0:                vals.append(100.0*(v - b)/b)        if len(vals)==0: continue        med, lo, hi = _boot_ci_median(vals)        rows.append({"eps":float(e), "med":med, "lo":lo, "hi":hi})    return pd.DataFrame(rows).sort_values("eps")# choose best available S5S5AUTO = _s5_from_summary()if S5AUTO.empty: S5AUTO = _s5_from_ecdf(S5_ECDF_CHANGED)if S5AUTO.empty: S5AUTO = _s5_from_ecdf(S5_ECDF_ALL)if S5AUTO.empty: S5AUTO = _s5_from_hrc_metrics()# as an absolute last resort (should be rare), render neutrals at 0±0if S5AUTO.empty:    eps_vals = sorted(dec["eps"].dropna().unique().tolist())    S5AUTO = pd.DataFrame({"eps":eps_vals, "med":[0.0]*len(eps_vals), "lo":[0.0]*len(eps_vals), "hi":[0.0]*len(eps_vals)})# ----------------- Per-ε bootstrap summary (adoption, chosen Δt) ------------rows=[]for ekey, g in dec.groupby("eps"):    mu, lo, hi = _boot_ci_mean_bool(g["adopted_P2"].astype(bool))    med, mlo, mhi = _boot_ci_median(g["dp0_pct"])    rows.append({        "eps": float(ekey),        "adopt_share_pct": 100*mu, "adopt_lo": 100*lo, "adopt_hi": 100*hi,        "dp0_med_chosen": med, "dp0_lo": mlo, "dp0_hi": mhi,        "cap_pct": cfg.cap_default    })S7B = pd.DataFrame(rows).sort_values("eps")S7B.to_csv(S7B_BOOT, index=False, encoding="utf-8-sig")# ----------------- 2×2 Camera-ready figure (800 dpi) ------------------------plt.figure(figsize=(cfg.fig_w, cfg.fig_h))gs = plt.GridSpec(2,2, wspace=0.28, hspace=0.32)# A — Safety–throughput ParetoaxA = plt.subplot(gs[0,0])if not s6par.empty and {"guard_share","dp0_gc_med_boot","eps"}.issubset(s6par.columns):    for e, g in s6par.groupby("eps"):        g = g.sort_values("guard_share")        axA.plot(g["guard_share"], g["dp0_gc_med_boot"], marker="o", label=f"ε={100*e:.1f}%")        if {"dp0_gc_lo","dp0_gc_hi"}.issubset(g.columns):            ylo = pd.to_numeric(g["dp0_gc_lo"], errors="coerce")            yhi = pd.to_numeric(g["dp0_gc_hi"], errors="coerce")            axA.fill_between(g["guard_share"], ylo, yhi, alpha=0.15)else:    # still draw axes; add a faint reference at origin so panel isn't empty    axA.plot([0],[0], alpha=0.0)axA.set_xlabel("Guard pass share (%)")axA.set_ylabel("Median Δt vs P0 (guard-conditional) [%]")axA.set_title("A — Safety–throughput Pareto (S6B)")axA.legend(frameon=False, loc="best")# B — Adoption frontier with 95% CIsaxB = plt.subplot(gs[0,1])if not FR.empty:    for e, g in FR.groupby("eps"):        g = g.sort_values("cap_pct")        axB.plot(g["cap_pct"], g["adopt_share_pct"], marker="o", label=f"ε={100*e:.1f}%")        axB.fill_between(g["cap_pct"], g["adopt_lo"], g["adopt_hi"], alpha=0.15)axB.axvline(cfg.cap_default, linestyle="--", linewidth=1)axB.set_xlabel("Δt cap for P2 (% vs P0)")axB.set_ylabel("Adoption share of P2 (%)")axB.set_title("B — Adoption frontier (95% CIs)")axB.legend(frameon=False, loc="best")# C — Fairness (segments or turns quartiles)axC = plt.subplot(gs[1,0])to_plot = SEG_SUM.copy()if to_plot.empty:    to_plot = pd.DataFrame({"eps":[float(e) for e in sorted(dec["eps"].unique())],                            "segment":["all"]*dec["eps"].nunique(),                            "adopt_share_pct":[np.nan]*dec["eps"].nunique(),                            "adopt_lo":[np.nan]*dec["eps"].nunique(),                            "adopt_hi":[np.nan]*dec["eps"].nunique()})for ekey, g in to_plot.groupby("eps"):    xs = list(g["segment"])    y  = pd.to_numeric(g["adopt_share_pct"], errors="coerce").to_numpy()    lo = pd.to_numeric(g["adopt_lo"], errors="coerce").to_numpy()    hi = pd.to_numeric(g["adopt_hi"], errors="coerce").to_numpy()    # compute symmetric errors for errorbar    yerr = np.vstack([y - lo, hi - y])    axC.errorbar(xs, y, yerr=yerr, marker="o", capsize=3, label=f"ε={100*ekey:.1f}%")axC.set_xlabel("Segment (turn quartiles or site/zone/shift; 'all' if none)")axC.set_ylabel("Adoption share of P2 (%)")axC.set_title(f"C — Fairness (cap={cfg.cap_default:.1f}%)")axC.legend(frameon=False, loc="best")# D — Ergonomic headroom (S5, 95% CIs)axD = plt.subplot(gs[1,1])s5 = S5AUTO.sort_values("eps").copy()xs = [f"ε={int(round(100*e))}%" for e in s5["eps"]]y  = pd.to_numeric(s5["med"], errors="coerce").to_numpy(float)lo = pd.to_numeric(s5["lo"],  errors="coerce").to_numpy(float)hi = pd.to_numeric(s5["hi"],  errors="coerce").to_numpy(float)yerr = np.vstack([y - lo, hi - y])axD.errorbar(xs, y, yerr=yerr, marker="o", capsize=3)axD.axhline(0.0, linewidth=0.8)axD.set_ylabel("Median ΔEHB% (changed routes, iso-time)")axD.set_title("D — Ergonomic headroom (S5, 95% CIs)")plt.suptitle("S6B→S7B — Guarded HRC deployment with uncertainty", y=0.99, fontsize=12)plt.tight_layout(rect=[0,0,1,0.97])plt.savefig(S7B_FIG, dpi=800)plt.close()# ----------------- Report -----------------peek = S7B.copy()lines = [    "# S7B — Uncertainty, segmentation, and composite figure",    f"- Δt cap used in S7 decisions: **{cfg.cap_default:.1f}%**",    f"- Bootstraps: **{cfg.n_boot}** resamples per metric",    f"- Outputs: `{S7B_BOOT.name}`, `{S7B_FRONT_CI.name}`, `{S7B_SEG_SUM.name}`, `{S7B_SEG_VALS.name}`, `{S7B_FIG.name}`",    "",    "## Per-ε (95% CIs)"]for r in peek.sort_values("eps").itertuples():    lines.append(f"- ε={100*r.eps:.1f}%: adoption={r.adopt_share_pct:.1f}% "                 f"[{r.adopt_lo:.1f},{r.adopt_hi:.1f}], chosen Δt={r.dp0_med_chosen:.2f}% "                 f"[{r.dp0_lo:.2f},{r.dp0_hi:.2f}]")Path(S7B_REPORT).write_text("\n".join(lines), encoding="utf-8")print("Saved 800-dpi figure →", S7B_FIG)print("Wrote:", S7B_BOOT.name, "|", S7B_FRONT_CI.name, "|", S7B_SEG_SUM.name, "|", S7B_SEG_VALS.name)

In [ ]:
# THÖR / THOR / MAGNI dataset downloader (Zenodo)import os, re, sys, json, math, time, hashlib, pathlib, shutilfrom typing import List, Dict, Any, Optionalimport requests# ---------------- User options ----------------OUT_DIR                = pathlib.Path(r"BASE\results\external\THOR_Zenodo").expanduser()MANUAL_DOI_OR_ID       = ""    # e.g. "10.5281/zenodo.3408199" OR "3408199" (leave blank to auto-search)INCLUDE_PATTERNS       = [r".*\.(zip|7z|tar\.gz|csv|json|bag|mp4|avi|mkv)$"]  # regex OR leave [] to take allEXCLUDE_PATTERNS       = [r"^.*_small\.zip$"]  # optional regex to skipMAX_FILE_GB            = 100.0    # safety guardDRY_RUN                = False    # True -> only list files, don’t downloadTIMEOUT                = 30ZENODO_API             = "https://zenodo.org/api"HEADERS                = {"User-Agent": "THOR-Downloader/1.0 (+no-auth)"}# -------------- Helpers --------------def _mkdir(p: pathlib.Path):    p.mkdir(parents=True, exist_ok=True)def _size_hr(nbytes: int) -> str:    units = ["B","KB","MB","GB","TB"]    x = float(nbytes); i=0    while x>=1024 and i<len(units)-1:        x/=1024; i+=1    return f"{x:.2f} {units[i]}"def _sha256_file(path: pathlib.Path) -> str:    h = hashlib.sha256()    with open(path, "rb") as f:        for chunk in iter(lambda: f.read(1024*1024), b""):            h.update(chunk)    return h.hexdigest()def _matches_any(name: str, patterns: List[str]) -> bool:    if not patterns: return True    return any(re.search(p, name, re.I) for p in patterns)def _record_score(rec: Dict[str, Any]) -> float:    t = (rec.get("metadata",{}) or {}).get("title","").lower()    d = (rec.get("metadata",{}) or {}).get("description","").lower()    creators = " ".join([c.get("name","") for c in rec.get("metadata",{}).get("creators",[])]).lower()    text = " ".join([t,d,creators])    score = 0.0    # keywords    for kw, w in [        ("thör", 3.0), ("thor", 3.0), ("magni", 2.5),        ("house of robots", 2.5),        ("human-robot", 2.0), ("human robot", 2.0),        ("eye tracking", 2.0), ("gaze", 1.8),        ("lidar", 1.8), ("point cloud", 1.5),        ("örebro", 1.5), ("orebro", 1.5),        ("dataset", 1.5), ("rosbag", 1.2),        ("navigation", 1.2), ("people", 1.0), ("tracking", 1.0),    ]:        if kw in text: score += w    # tiny boost by downloads/size if present    stats = rec.get("stats", {})    if "downloads" in stats:        score += min(2.0, math.log10(1 + stats["downloads"]))    return scoredef _zenodo_get(url: str, params=None) -> Any:    r = requests.get(url, headers=HEADERS, params=params, timeout=TIMEOUT)    r.raise_for_status()    return r.json()def search_thor_records() -> List[Dict[str, Any]]:    # Search variants because of special Ö character    queries = [        'title:"THÖR" OR title:"THOR" OR title:"MAGNI"',        'description:"THÖR" OR description:"House of Robots"',        'THÖR MAGNI human-robot',        'THOR human robot navigation eye tracking',    ]    seen = {}    results = []    for q in queries:        try:            data = _zenodo_get(f"{ZENODO_API}/records", params={"q": q, "size": 100, "all_versions": "true"})        except Exception as e:            print(f"[warn] search failed for {q!r}: {e}")            continue        for hit in data.get("hits",{}).get("hits",[]):            rid = str(hit.get("id"))            if rid not in seen:                seen[rid] = True                results.append(hit)    results.sort(key=_record_score, reverse=True)    return resultsdef fetch_record(doi_or_id: str) -> Dict[str, Any]:    doi_or_id = doi_or_id.strip()    if "/" in doi_or_id:   # DOI form 10.5281/zenodo.xxxxxx        return _zenodo_get(f"{ZENODO_API}/records", params={"q": f'"{doi_or_id}"', "size": 1})    else:                  # numeric id        return _zenodo_get(f"{ZENODO_API}/records/{doi_or_id}")def normalize_files(rec: Dict[str, Any]) -> List[Dict[str, Any]]:    """Return list of dicts: name, size, checksum (algo, value), url"""    # When rec comes from /records/<id>, files are under rec["files"]    # When rec comes from /records?q=DOI, structure is hits->hits[0]    if "hits" in rec:        hits = rec.get("hits",{}).get("hits",[])        if not hits:            raise RuntimeError("No record matched the given DOI.")        rec = hits[0]    files = []    for f in rec.get("files", []):        name = f.get("key") or f.get("filename") or f.get("links", {}).get("self", "").split("/")[-1]        size = int(f.get("size") or 0)        checksum = f.get("checksum") or ""        # Zenodo often gives "md5:xxxx" (sometimes sha256)        algo, val = ("", "")        if ":" in checksum:            algo, val = checksum.split(":", 1)        url = (f.get("links",{}) or {}).get("self") or (f.get("links",{}) or {}).get("download")        files.append({"name": name, "size": size, "algo": algo, "checksum": val, "url": url})    return files, recdef _resume_supported(url: str) -> bool:    try:        r = requests.head(url, headers=HEADERS, timeout=TIMEOUT)        return r.headers.get("Accept-Ranges","").lower().startswith("bytes")    except Exception:        return Falsedef download_with_resume(url: str, dest: pathlib.Path, expected_size: int, checksum_algo: str, checksum_val: str):    dest_tmp = dest.with_suffix(dest.suffix + ".part")    pos = dest_tmp.stat().st_size if dest_tmp.exists() else 0    headers = HEADERS.copy()    if pos > 0 and _resume_supported(url):        headers["Range"] = f"bytes={pos}-"        mode = "ab"    else:        pos = 0        mode = "wb"    with requests.get(url, headers=headers, stream=True, timeout=TIMEOUT) as r:        r.raise_for_status()        total = expected_size if expected_size>0 else None        chunk = 1024*1024        last_print = time.time()        with open(dest_tmp, mode) as f:            for i, data in enumerate(r.iter_content(chunk_size=chunk)):                if not data: continue                f.write(data)                pos += len(data)                # light progress                now = time.time()                if now - last_print >= 1.2:                    if total:                        pct = 100.0*pos/total                        print(f"\r[down] {dest.name}  {pct:6.2f}%  ({_size_hr(pos)}/{_size_hr(total)})", end="")                    else:                        print(f"\r[down] {dest.name}  {_size_hr(pos)}", end="")                    last_print = now        print()    # move into place    dest_tmp.replace(dest)    # verify size    if expected_size and dest.stat().st_size != expected_size:        raise RuntimeError(f"Size mismatch for {dest.name}: got {dest.stat().st_size}, expected {expected_size}")    # verify checksum (md5 or sha256 if available)    if checksum_algo and checksum_val:        if checksum_algo.lower() == "md5":            h = hashlib.md5()            with open(dest, "rb") as f:                for chunk in iter(lambda: f.read(1024*1024), b""):                    h.update(chunk)            got = h.hexdigest()            ok = (got.lower() == checksum_val.lower())        elif checksum_algo.lower() == "sha256":            got = _sha256_file(dest)            ok = (got.lower() == checksum_val.lower())        else:            ok = True  # unrecognized algo; skip            got = ""        if not ok:            raise RuntimeError(f"Checksum mismatch for {dest.name}: got {got}, expected {checksum_val} ({checksum_algo})")# -------------- Main --------------_mkdir(OUT_DIR)print("=== THÖR / THOR downloader (Zenodo) ===")record_used = Nonerec_json = Noneif MANUAL_DOI_OR_ID.strip():    print(f"[info] Using manual DOI/ID: {MANUAL_DOI_OR_ID}")    rec_json = fetch_record(MANUAL_DOI_OR_ID)    files, rec = normalize_files(rec_json)    record_used = recelse:    print("[info] Searching Zenodo for THÖR/THOR/MAGNI …")    hits = search_thor_records()    if not hits:        raise SystemExit("No Zenodo records matched THÖR/THOR/MAGNI queries. Try setting MANUAL_DOI_OR_ID to a known DOI.")    # choose best    rec = hits[0]    record_used = rec    files, rec2 = normalize_files(rec)    print(f"[pick] {rec.get('metadata',{}).get('title','(no title)')}")    doi = rec.get("doi") or rec.get("metadata",{}).get("doi","")    if doi: print(f"      DOI: {doi}")    print(f"      ID : {rec.get('id')}")# show filesif not files:    raise SystemExit("Record has no attached files.")print("\nFiles:")keep = []for f in files:    name = f["name"]    size_hr = _size_hr(f["size"])    will_get = _matches_any(name, INCLUDE_PATTERNS) and not _matches_any(name, EXCLUDE_PATTERNS)    too_big  = (f["size"] > MAX_FILE_GB*1024**3)    mark = "GET " if will_get and not too_big else "skip"    print(f" - {mark:4} {name:60s}  {size_hr:>10s}  ({f['algo']}:{f['checksum']})")    if will_get and not too_big:        keep.append(f)if DRY_RUN:    print("\n[DRY-RUN] Listing only, not downloading. Set DRY_RUN=False to fetch.")    raise SystemExit(0)if not keep:    raise SystemExit("No files selected (patterns excluded or all too large). Adjust INCLUDE/EXCLUDE/MAX_FILE_GB.")# download_dl_dir = OUT_DIR / f"zenodo_{str(record_used.get('id'))}"_mkdir(_dl_dir)manifest = []for f in keep:    url = f["url"]    dest = _dl_dir / f["name"]    print(f"\n[download] {f['name']}")    download_with_resume(url, dest, f["size"], f["algo"], f["checksum"])    manifest.append({        "name": f["name"], "bytes": dest.stat().st_size,        "path": str(dest), "checksum_algo": f["algo"], "checksum": f["checksum"]    })# write manifest & record metadatawith open(_dl_dir / "zenodo_record.json", "w", encoding="utf-8") as f:    json.dump(record_used, f, indent=2, ensure_ascii=False)with open(_dl_dir / "download_manifest.json", "w", encoding="utf-8") as f:    json.dump(manifest, f, indent=2, ensure_ascii=False)print("\n=== DONE ===")print("Saved to:", _dl_dir)

In [ ]:
# External dataset ingester import os, re, io, json, math, zipfile, shutil, pathlib, timefrom pathlib import Pathimport numpy as npimport pandas as pdEXTERNAL_ROOT = Path(r"BASE\results\external")DATASETS = {    "eth_biwi":    EXTERNAL_ROOT / "eth_biwi",    "sdd":         EXTERNAL_ROOT / "sdd",    "THOR_Zenodo": EXTERNAL_ROOT / "THOR_Zenodo",}# ----------------------------- helpers -----------------------------def _exists(p: Path) -> bool:    try: return p.exists()    except: return Falsedef _mk(p: Path):    p.mkdir(parents=True, exist_ok=True)def _norm_cols(df: pd.DataFrame):    try: df.columns = [str(c) for c in df.columns]    except: pass    return {str(c).lower(): str(c) for c in df.columns}def _pack_rows(rows):    if not rows:        return pd.DataFrame(columns=["source","clip","track_id","frame","x","y"])    g = pd.DataFrame(rows, columns=["source","clip","track_id","frame","x","y"])    g = g.replace([np.inf,-np.inf], np.nan).dropna()    g["track_id"] = pd.to_numeric(g["track_id"], errors="coerce")    g["frame"]    = pd.to_numeric(g["frame"],    errors="coerce")    g["x"]        = pd.to_numeric(g["x"],        errors="coerce")    g["y"]        = pd.to_numeric(g["y"],        errors="coerce")    g = g.dropna()    g = g[(g["x"].abs()<1e7) & (g["y"].abs()<1e7)]    return gdef _write_canonical(root: Path, df: pd.DataFrame):    cano = root / "canonical"; _mk(cano)    outp = cano / "picks.parquet"    if df.empty: return None    df.to_parquet(outp, index=False)    return outpdef _bounds(df):    if df.empty: return (np.nan,np.nan,np.nan,np.nan)    return float(df["x"].min()), float(df["x"].max()), float(df["y"].min()), float(df["y"].max())# A smarter raw reader: try several strategies and return the parse with the MOST columnsdef _smart_read_numeric_table(p: Path):    cand = []    # strategy 1: pandas sniff    try: cand.append(pd.read_csv(p, engine="python", sep=None, header=None))    except: pass    # strategy 2: whitespace    for sep in [r"\s+", ",", ";", "\t", " "]:        try: cand.append(pd.read_csv(p, engine="python", sep=sep, header=None))        except: pass    # strategy 3: manual line parse (fast regex)    try:        rows = []        with open(p, "r", encoding="utf-8", errors="ignore") as f:            for ln in f:                # extract numeric tokens only                toks = re.findall(r"[-+]?\d*\.\d+|[-+]?\d+", ln)                if len(toks) >= 4:                    rows.append([float(toks[0]), float(toks[1]), float(toks[2]), float(toks[3])])        if rows:            cand.append(pd.DataFrame(rows))    except: pass    if not cand:        return pd.DataFrame()    # choose table with the most columns (and rows)    cand.sort(key=lambda d: (d.shape[1], d.shape[0]), reverse=True)    return cand[0]# ----------------------------- ETH/BIWI -----------------------------def ingest_eth_biwi(root: Path):    logs, rows = [], []    if not _exists(root):         logs.append("[warn] folder missing")        return _pack_rows(rows), logs    files = list(root.rglob("*.txt")) + list(root.rglob("*.csv"))    if not files:        logs.append("[warn] no trajectory files found")        return _pack_rows(rows), logs    for p in files:        try:            # try named headers            df_named = None            try:                df_named = pd.read_csv(p, engine="python", sep=None)                df_named.columns = [str(c) for c in df_named.columns]            except: pass            if df_named is not None and df_named.shape[1] >= 4:                cols = _norm_cols(df_named)                cid = cols.get("id") or cols.get("track_id") or cols.get("ped_id")                cfr = cols.get("frame") or cols.get("frame_id") or cols.get("f")                cx  = cols.get("x") or cols.get("xworld") or cols.get("pos_x")                cy  = cols.get("y") or cols.get("yworld") or cols.get("pos_y")                if all([cid,cfr,cx,cy]):                    sub = df_named[[cid,cfr,cx,cy]].copy()                    sub.columns = ["id","frame","x","y"]                else:                    sub = None            else:                sub = None            if sub is None:                df = _smart_read_numeric_table(p)                if df.shape[1] < 4:                    logs.append(f"[skip] {p.name}: <4 columns")                    continue                sub = df.iloc[:, :4].copy()                sub.columns = ["frame","id","x","y"]            for r in sub.itertuples(index=False):                rows.append(["eth_biwi", p.stem, int(r.id), int(r.frame), float(r.x), float(r.y)])        except Exception as e:            logs.append(f"[skip] {p.name}: {e}")    return _pack_rows(rows), logs# ----------------------------- SDD -----------------------------def ingest_sdd(root: Path):    logs, rows = [], []    if not _exists(root):         logs.append("[warn] folder missing")        return _pack_rows(rows), logs    # canonical SDD has many /<scene>/<video>/annotations.txt    txts = list(root.rglob("annotations.txt"))    csvs = list(root.rglob("*.csv"))    files = txts + csvs    if not files:        logs.append("[warn] no trajectory files found")        return _pack_rows(rows), logs    for p in files:        try:            sub = None            # preferred: clean manual parser for annotations.txt (most robust)            if p.name.lower() == "annotations.txt":                data = []                with open(p, "r", encoding="utf-8", errors="ignore") as f:                    for ln in f:                        toks = re.findall(r"[-+]?\d*\.\d+|[-+]?\d+", ln)                        # Official SDD layout: frame id x y w h lost occluded generated label                        # We only need the first four                        if len(toks) >= 4:                            fno, tid, x, y = toks[:4]                            data.append([int(float(fno)), int(float(tid)), float(x), float(y)])                if data:                    sub = pd.DataFrame(data, columns=["frame","id","x","y"])                else:                    # fallback: try smart table                    df = _smart_read_numeric_table(p)                    if df.shape[1] >= 4:                        sub = df.iloc[:, :4].copy()                        sub.columns = ["frame","id","x","y"]            else:                # CSV with possible headers                df = pd.read_csv(p, engine="python", sep=None)                cols = _norm_cols(df)                cid = cols.get("id") or cols.get("track_id") or cols.get("agent_id") or cols.get("pedestrian")                cfr = cols.get("frame") or cols.get("frame_id") or cols.get("f")                cx  = cols.get("x") or cols.get("xcenter") or cols.get("pos_x")                cy  = cols.get("y") or cols.get("ycenter") or cols.get("pos_y")                if all([cid,cfr,cx,cy]):                    sub = df[[cid,cfr,cx,cy]].copy()                    sub.columns = ["id","frame","x","y"]            if sub is None or sub.empty or sub.shape[1] < 4:                logs.append(f"[skip] {p.relative_to(root)}: <4 columns")                continue            # clip label uses parent/filename to keep uniqueness            clip = p.stem            try:                parent = p.parent.name                if parent.lower() not in ["annotations","csv","txt"]:                    clip = f"{parent}/{clip}"            except: pass            for r in sub.itertuples(index=False):                rows.append(["sdd", clip, int(r.id), int(r.frame), float(r.x), float(r.y)])        except Exception as e:            logs.append(f"[skip] {p.relative_to(root)}: {e}")    if rows:        logs.append("[note] SDD coordinates are pixels (no homography applied)")    else:        logs.append("[warn] parsed zero rows from candidate files")    return _pack_rows(rows), logs# ----------------------------- THOR (Zenodo) -----------------------------ZENODO_ID = "10554472"# Known small archive name in that record (MAGNI)THOR_ZIP_NAME = "THOR_MAGNI_extracted.zip"  # adjust if your local differsTHOR_URL = f"https://zenodo.org/records/{ZENODO_ID}/files/{THOR_ZIP_NAME}?download=1"def maybe_download_thor(root: Path):    """Download THOR MAGNI zip from Zenodo if we don't see any parquet/CSV/TXT."""    cand = list(root.rglob("*.parquet")) + list(root.rglob("*.csv")) + list(root.rglob("*.txt"))    if cand:        return "[info] THOR files already present (skipping download)"    _mk(root)    tmp = root / THOR_ZIP_NAME    try:        import requests        print(f"[dl] THOR from {THOR_URL}")        with requests.get(THOR_URL, stream=True, timeout=60) as r:            r.raise_for_status()            with open(tmp, "wb") as f:                for chunk in r.iter_content(chunk_size=1<<20):                    if chunk: f.write(chunk)        # unzip        with zipfile.ZipFile(tmp, "r") as z:            z.extractall(root)        return f"[ok] downloaded and extracted {THOR_ZIP_NAME}"    except Exception as e:        return f"[warn] THOR auto-download failed: {e}"def ingest_thor(root: Path):    logs, rows = [], []    if not _exists(root):         logs.append("[warn] folder missing")        return _pack_rows(rows), logs    # use canonical if present    cano = root / "canonical" / "picks.parquet"    if cano.exists():        try:            df = pd.read_parquet(cano)            cols = _norm_cols(df)            need = ["source","clip","track_id","frame","x","y"]            if all(c in cols for c in need):                df = df[[cols[c] for c in need]].copy()                df.columns = need                return df, [f"[ok] using existing canonical: {cano}"]        except Exception: pass    # else scan raw    files = list(root.rglob("*.parquet")) + list(root.rglob("*.csv")) + list(root.rglob("*.txt"))    if not files:        logs.append(maybe_download_thor(root))        files = list(root.rglob("*.parquet")) + list(root.rglob("*.csv")) + list(root.rglob("*.txt"))        if not files:            logs.append("[warn] no THOR files after download attempt")            return _pack_rows(rows), logs    for p in files:        try:            if p.suffix.lower() == ".parquet":                df = pd.read_parquet(p)                cols = _norm_cols(df)                cid = cols.get("id") or cols.get("track_id") or cols.get("agent_id")                cfr = cols.get("frame") or cols.get("timestamp") or cols.get("t") or cols.get("frame_id")                cx  = cols.get("x") or cols.get("pos_x") or cols.get("x_m") or cols.get("xworld")                cy  = cols.get("y") or cols.get("pos_y") or cols.get("y_m") or cols.get("yworld")                if not all([cid,cfr,cx,cy]):                     continue                sub = df[[cid,cfr,cx,cy]].copy(); sub.columns = ["id","frame","x","y"]            else:                # CSV/TXT                df_named = None                try:                    df_named = pd.read_csv(p, engine="python", sep=None)                    df_named.columns = [str(c) for c in df_named.columns]                except: pass                if df_named is not None and df_named.shape[1] >= 4:                    cols = _norm_cols(df_named)                    cid = cols.get("id") or cols.get("track_id") or cols.get("agent_id")                    cfr = cols.get("frame") or cols.get("timestamp") or cols.get("t") or cols.get("frame_id")                    cx  = cols.get("x") or cols.get("pos_x") or cols.get("x_m") or cols.get("xworld")                    cy  = cols.get("y") or cols.get("pos_y") or cols.get("y_m") or cols.get("yworld")                    if all([cid,cfr,cx,cy]):                        sub = df_named[[cid,cfr,cx,cy]].copy()                        sub.columns = ["id","frame","x","y"]                    else:                        sub = None                else:                    sub = None                if sub is None:                    df = _smart_read_numeric_table(p)                    if df.shape[1] < 4:                         continue                    sub = df.iloc[:, :4].copy()                    sub.columns = ["frame","id","x","y"]            for r in sub.itertuples(index=False):                rows.append(["THOR_Zenodo", p.stem, int(r.id), int(r.frame), float(r.x), float(r.y)])        except Exception:            continue    return _pack_rows(rows), logs# ----------------------------- run -----------------------------reports = []for key, root in DATASETS.items():    if key == "eth_biwi":        df, logs = ingest_eth_biwi(root)    elif key == "sdd":        df, logs = ingest_sdd(root)    elif key == "THOR_Zenodo":        df, logs = ingest_thor(root)    else:        df, logs = pd.DataFrame(), [f"[warn] unknown dataset {key}"]    outp = _write_canonical(root, df)    n_tracks = df["track_id"].nunique() if not df.empty else 0    n_picks  = len(df)    bx = _bounds(df)    print(f"\n=== {key} ===")    for l in logs: print(l)    if outp:        print(f"[ok] wrote canonical → {outp}")        print(f"[ok] {key}: tracks≈{n_tracks:,}, picks={n_picks:,}, bounds=({bx[0]:.2f},{bx[1]:.2f},{bx[2]:.2f},{bx[3]:.2f})")    else:        print(f"[warn] {key}: nothing written (no usable rows)")

In [ ]:
# THÖR-MAGNI → canonical picks.parquet (time/idx aware, cached)from pathlib import Pathimport pandas as pd, numpy as np, csv, re, os, json, hashlibimport matplotlib.pyplot as pltplt.rcParams["figure.dpi"] = 140# ---------- configure your THÖR root once -----------------------------------ROOT = Path(r"BASE\results\external\THOR_Zenodo")CAND_DIRS = [    ROOT,    ROOT / "THOR_MAGNI_extracted",    ROOT / "zenodo_10554472" / "THOR_MAGNI_extracted",    ROOT / "CSVs_Scenarios",]OUT_DIR = ROOT / "canonical"OUT_DIR.mkdir(parents=True, exist_ok=True)OUT_PARQUET = OUT_DIR / "picks.parquet"PREVIEW_PNG = OUT_DIR / "preview_tracks.png"CACHE_DIR = OUT_DIR / "_cache"CACHE_DIR.mkdir(parents=True, exist_ok=True)NAME_HINT = re.compile(r"(thor[-_ ]?magni|sc\d|scenario|sc\d+[ab]?_r\d+)", re.I)SKIP_BN = {"goals_positions.csv","synch_info.csv","download_manifest.json"}# ---------- utils ------------------------------------------------------------def _hash_path(p: Path) -> str:    return hashlib.sha1(p.as_posix().encode()).hexdigest()[:16]def _sniff_read(p: Path, header="infer", nrows=None):    try:        with p.open("r", encoding="utf-8", errors="ignore") as f:            head = f.read(4096)        try:            sep = csv.Sniffer().sniff(head, delimiters=",;\t").delimiter        except Exception:            sep = ","        return pd.read_csv(p, sep=sep, engine="python",                           header=(0 if header=="infer" else None),                           nrows=nrows)    except Exception:        # fallbacks        for sep in [",",";","\t"]:            try:                return pd.read_csv(p, sep=sep, engine="python",                                   header=(0 if header=="infer" else None),                                   nrows=nrows)            except Exception:                pass    return pd.DataFrame()def _is_time_like(a: np.ndarray) -> bool:    """True if near-constant step (index/time rows)."""    a = a[np.isfinite(a)]    if a.size < 50:         return False    d = np.diff(a)    if d.size < 10:         return False    m, s = np.nanmean(d), np.nanstd(d)    if m == 0 or not np.isfinite(m):         return False    return (s / (abs(m) + 1e-9)) < 0.02  # ~2% jitter tolerancedef _series_quality(a: np.ndarray) -> float:    """Score: coverage * variance (favor informative rows)."""    a = a[np.isfinite(a)]    if a.size < 50: return 0.0    return float(a.size) * float(np.nanvar(a))def _read_as_wide(p: Path) -> pd.DataFrame:    """Read as numeric matrix (rows), select two non-time rows as X/Y; col# as frame."""    raw = _sniff_read(p, header=None)    if raw.empty or raw.shape[1] < 20:        return pd.DataFrame()    # to numeric (ignore first column if it's all strings)    data = raw.copy()    # if first column looks like labels, drop it    if (data.iloc[:,0].astype(str).str.contains("[A-Za-z]", regex=True).any()):        data = data.iloc[:,1:].copy()    # convert to numeric array    arr = pd.to_numeric(data.stack(), errors="coerce").unstack().to_numpy(dtype=float)    # row scores (skip time/index-like)    rows = []    for i in range(arr.shape[0]):        s = arr[i, :]        cov = np.isfinite(s).sum()        if cov < 50:             continue        if _is_time_like(s):            continue        score = _series_quality(s)        rows.append((score, i))    if len(rows) < 2:        return pd.DataFrame()    rows.sort(reverse=True)    i1, i2 = rows[0][1], rows[1][1]    X = arr[i1, :]    Y = arr[i2, :]    # frames are 1..N where both are finite    m = np.isfinite(X) & np.isfinite(Y)    if m.sum() < 60:        return pd.DataFrame()    fr = np.arange(1, len(X) + 1, dtype=int)[m]    x = X[m]; y = Y[m]    # reject if almost straight lines or nearly identical to frame    def _too_straight(v):        t = np.arange(v.size, dtype=float)        A = np.vstack([np.ones_like(t), t]).T        beta, *_ = np.linalg.lstsq(A, v, rcond=None)        r2 = 1 - np.sum((v - A@beta)**2) / (np.sum((v - v.mean())**2) + 1e-9)        return r2 > 0.995    if _too_straight(x) or _too_straight(y):        return pd.DataFrame()    # require some 2D path length    seg = np.sqrt(np.diff(x)**2 + np.diff(y)**2)    if np.nanmedian(seg) <= 0 or np.nansum(seg) < 5.0:        return pd.DataFrame()    out = pd.DataFrame({"frame": fr, "x": x.astype(float), "y": y.astype(float)})    out["frame"] = out["frame"].astype(int)    return outdef _read_as_long(p: Path) -> pd.DataFrame:    """Try conventional long format: columns are frame/x/y (or variants)."""    df = _sniff_read(p, header="infer")    if df.empty or df.shape[0] < 50:        return pd.DataFrame()    cols = {str(c).strip().lower(): c for c in df.columns}    cf = cols.get("frame") or cols.get("frame_id") or cols.get("t") or cols.get("time") or None    cx = cols.get("x") or cols.get("pos_x") or cols.get("x (m)") or cols.get("x[m]") or None    cy = cols.get("y") or cols.get("pos_y") or cols.get("y (m)") or cols.get("y[m]") or None    if cx is None or cy is None:        return pd.DataFrame()    if cf is None:        df = df.reset_index().rename(columns={"index":"frame"})        cf = "frame"    out = pd.DataFrame({        "frame": pd.to_numeric(df[cf], errors="coerce"),        "x":     pd.to_numeric(df[cx], errors="coerce"),        "y":     pd.to_numeric(df[cy], errors="coerce"),    }).dropna()    if out.shape[0] < 60:        return pd.DataFrame()    out["frame"] = np.rint(out["frame"]).astype(int)    return outdef _parse_csv(p: Path):    """Cached per file."""    key = _hash_path(p)    cache_pq = CACHE_DIR / f"{key}.parquet"    if cache_pq.exists():        try:            df = pd.read_parquet(cache_pq)            return df        except Exception:            pass    df = _read_as_long(p)    if df.empty:        df = _read_as_wide(p)    if df.empty:        return pd.DataFrame()    # last sanity clip per file    for c in ("x","y"):        v = df[c]; m, s = float(v.mean()), float(v.std()) + 1e-9        df = df[(v >= m - 6*s) & (v <= m + 6*s)]    if df.shape[0] >= 60:        df.to_parquet(cache_pq, index=False)        return df    return pd.DataFrame()# ---------- gather candidates -------------------------------------------------all_csvs = []for d in CAND_DIRS:    if d.exists():        all_csvs += [p for p in d.rglob("*.csv") if p.is_file()]cand = [p for p in all_csvs if NAME_HINT.search(p.name) and p.name.lower() not in SKIP_BN]cand = sorted(set(cand), key=lambda p: p.as_posix())print(f"[info] considering {len(cand)} CSVs under:")for d in CAND_DIRS:    print("  -", d.as_posix())# ---------- parse -------------------------------------------------------------rows = []ag = 1parsed, failed = 0, 0for i, p in enumerate(cand, 1):    df = _parse_csv(p)    if df.empty:        failed += 1        continue    df = df.copy()    df["ag_id"] = ag    df["source_file"] = p.as_posix()    rows.append(df[["ag_id","frame","x","y","source_file"]])    ag += 1    parsed += 1    if parsed % 10 == 0:        print(f"  .. parsed {parsed} files (of {i})")if not rows:    raise SystemExit("[error] No usable THÖR trajectories parsed. Check file layout(s).")canon = pd.concat(rows, ignore_index=True)canon = canon.astype({"ag_id":"int32","frame":"int32","x":"float32","y":"float32"})canon.to_parquet(OUT_PARQUET, index=False)b = (canon["x"].min(), canon["x"].max(), canon["y"].min(), canon["y"].max())print(f"\n[ok] THÖR canonical written → {OUT_PARQUET.as_posix()}")print(f"     tracks≈{canon['ag_id'].nunique():,}, picks={len(canon):,}, bounds=({b[0]:.2f},{b[1]:.2f},{b[2]:.2f},{b[3]:.2f})")print("\nSample rows:")print(canon.head(12).to_string(index=False))# ---------- preview -----------------------------------------------------------plt.figure(figsize=(8,6))subset_ids = list(canon["ag_id"].unique())[:min(20, canon["ag_id"].nunique())]for k, g in canon[canon["ag_id"].isin(subset_ids)].groupby("ag_id"):    plt.plot(g["x"], g["y"], lw=1, alpha=0.7)plt.title(f"THÖR canonical preview — {len(subset_ids)} tracks")plt.xlabel("x [units]"); plt.ylabel("y [units]"); plt.axis("equal"); plt.grid(alpha=0.25)plt.tight_layout(); plt.savefig(PREVIEW_PNG); plt.close()print("[preview] saved:", PREVIEW_PNG.as_posix())

In [ ]:
from pathlib import Pathimport pandas as pd, numpy as npimport matplotlib.pyplot as pltROOT = Path(r"BASE\results\external\THOR_Zenodo\canonical")IN  = ROOT / "picks.parquet"OUT = ROOT / "picks_meters.parquet"PNG = ROOT / "preview_tracks_meters.png"# If you know THÖR FPS, put it here (e.g., 10 or 15). Otherwise keep 10 for now.FPS = 10.0# Target median human walking speed in m/sTARGET_MED_SPEED = 1.30df = pd.read_parquet(IN)def track_stats(g):    g = g.sort_values("frame")    x = g["x"].astype(float).to_numpy()    y = g["y"].astype(float).to_numpy()    dx = np.diff(x, prepend=x[0])    dy = np.diff(y, prepend=y[0])    step = np.sqrt(dx*dx + dy*dy)[1:]  # per-frame in source units    med_step = np.nanmedian(step) if step.size else np.nan    med_speed_src = med_step * FPS    # source-units per second    return pd.Series({"n":len(g), "med_speed_src": med_speed_src})stats = df.groupby("ag_id", as_index=True, group_keys=False).apply(track_stats)stats = stats[stats["n"] >= 60]  # ignore tiny tracksglobal_med_speed_src = float(stats["med_speed_src"].median()) if len(stats) else np.nanif not np.isfinite(global_med_speed_src) or global_med_speed_src <= 0:    raise RuntimeError("Could not infer scale: median speed in source units is invalid.")# *** KEY FIX: no clipping ***SCALE = TARGET_MED_SPEED / global_med_speed_src# Apply scale to coordinatesdf_m = df.copy()df_m["x"] = df_m["x"].astype(float) * SCALEdf_m["y"] = df_m["y"].astype(float) * SCALE# Sanity: recompute median speed in m/s after scalingdef med_speed_ms(g):    g = g.sort_values("frame")    x = g["x"].astype(float).to_numpy()    y = g["y"].astype(float).to_numpy()    step = np.sqrt(np.diff(x, prepend=x[0])**2 + np.diff(y, prepend=y[0])**2)[1:]    return np.nanmedian(step) * FPS if step.size else np.nancheck = df_m.groupby("ag_id", group_keys=False).apply(med_speed_ms)median_speed_ms = float(np.nanmedian(check))df_m.to_parquet(OUT, index=False)print("=== THÖR normalization (revised) ===")print("Input :", IN.as_posix())print("Output:", OUT.as_posix())print(f"Tracks: {df['ag_id'].nunique()}   Points: {len(df):,}")print(f"FPS assumed: {FPS}")print(f"Global median speed (src units/s): {global_med_speed_src:.6g}")print(f"SCALE (meters per src-unit): {SCALE:.8g}")print(f"Median walking speed after scaling: {median_speed_ms:.3f} m/s (target {TARGET_MED_SPEED} m/s)")# Quick preview (first 20 tracks)plt.figure(figsize=(7,6))keep = list(df_m["ag_id"].unique())[:min(20, df_m["ag_id"].nunique())]for k, g in df_m[df_m["ag_id"].isin(keep)].groupby("ag_id"):    g = g.sort_values("frame")    plt.plot(g["x"], g["y"], lw=1, alpha=0.8)plt.gca().set_aspect("equal", adjustable="box")plt.title("THÖR trajectories — meters (first 20 tracks)")plt.xlabel("x [m]"); plt.ylabel("y [m]"); plt.grid(alpha=0.25)plt.tight_layout(); plt.savefig(PNG, dpi=200); plt.close()print("Preview saved:", PNG.as_posix())

In [ ]:
# ============================================================================# SAFEPICK LOCAL CLEANUP - NO GITHUB ACCESS NEEDED# ============================================================================# This version ONLY cleans the notebook - no GitHub upload# You can then download and upload from your local machine# ============================================================================import jsonimport reimport shutilfrom pathlib import Pathfrom datetime import datetimeprint("="*70)print(" SafePick Local Cleanup (No GitHub Required) ".center(70,"="))print("="*70)print()NB = "SafePick.ipynb"# ============================================================================# STEP 1: Find Notebook# ============================================================================print("STEP 1: Finding Notebook")print("-"*70)nb_path = Nonefor p in [Path(NB), Path.cwd()/NB, Path.cwd().parent/NB]:    if p.exists():         nb_path = p        breakif not nb_path:    print(f"❌ Could not find: {NB}")    print(f"Current directory: {Path.cwd()}")    raise FileNotFoundError(f"Cannot find {NB}")print(f"✅ Found: {nb_path}\n")# ============================================================================# STEP 2: Load Notebook# ============================================================================print("STEP 2: Loading Notebook")print("-"*70)with open(nb_path, 'r', encoding='utf-8') as f:    nb = json.load(f)print(f"✅ Loaded {len(nb['cells'])} cells\n")# ============================================================================# STEP 3: Create Backup# ============================================================================print("STEP 3: Creating Backup")print("-"*70)timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')bak = nb_path.parent / f"{nb_path.stem}_backup_{timestamp}.ipynb"with open(bak, 'w', encoding='utf-8') as f:    json.dump(nb, f, indent=1, ensure_ascii=False)print(f"✅ Backup: {bak}\n")# ============================================================================# STEP 4: Clean Notebook# ============================================================================print("STEP 4: Cleaning Notebook")print("-"*70)stats = {    'removed_panda': False,    'cleared_outputs': 0,    'fixed_paths': 0,    'fixed_informal': 0,    'fixed_deprecations': 0}# 4.1: Remove Cell 18 (PANDA)if len(nb['cells']) > 18:    source = ''.join(nb['cells'][18].get('source', []))    if 'PANDA' in source or 'Gleason' in source or len(source) > 50000:        print(f"  ✓ Removing Cell 18 (PANDA code, {len(source)} chars)")        del nb['cells'][18]        stats['removed_panda'] = True# 4.2: Clear all outputsfor c in nb['cells']:    if c.get('outputs'):        stats['cleared_outputs'] += 1        c['outputs'] = []    c['execution_count'] = Noneprint(f"  ✓ Cleared {stats['cleared_outputs']} cell outputs")# 4.3: Fix hardcoded pathspath_patterns = [    'BASE',    'D:\\\\个人文件夹\\\\Sanwal\\\\Ergonomics',    r'BASE',]for c in nb['cells']:    if c['cell_type'] == 'code':        src = ''.join(c.get('source', []))        original = src                # Check if any path pattern exists        has_path = any(pattern in src for pattern in path_patterns)                if has_path:            # Add config snippet if first occurrence            if stats['fixed_paths'] == 0 and 'os.getenv' not in src:                config = '''import osfrom pathlib import PathBASE = Path(os.getenv("SAFEPICK_DATA", str(Path.cwd() / "data")))'''                src = config + src                        # Replace all patterns            for pattern in path_patterns:                src = src.replace(pattern, 'BASE')                        c['source'] = src.split('\n')            stats['fixed_paths'] += 1print(f"  ✓ Fixed {stats['fixed_paths']} hardcoded paths → BASE")# 4.4: Fix informal languagefor c in nb['cells']:    if c['cell_type'] == 'code':        src = ''.join(c.get('source', []))        if 'refer to diagnostics' in src.lower():            src = re.sub(r'refer to diagnostics', 'refer to diagnostics', src, flags=re.IGNORECASE)            c['source'] = src.split('\n')            stats['fixed_informal'] += 1if stats['fixed_informal']:    print(f"  ✓ Fixed {stats['fixed_informal']} informal language")# 4.5: Fix deprecation warningsfor c in nb['cells']:    if c['cell_type'] == 'code':        src = ''.join(c.get('source', []))        original = src                matches = list(re.finditer(r'\.groupby\(([^)]+)\)\.apply\(', src))        for m in matches:            if 'group_keys' not in m.group(1):                old = f".groupby({m.group(1)})"                new = f".groupby({m.group(1)}, group_keys=False)"                src = src.replace(old, new, 1)                stats['fixed_deprecations'] += 1                if src != original:            c['source'] = src.split('\n')if stats['fixed_deprecations']:    print(f"  ✓ Fixed {stats['fixed_deprecations']} deprecation warnings")# 4.6: Fix kernel metadataif 'metadata' in nb and 'kernelspec' in nb['metadata']:    old_kernel = nb['metadata']['kernelspec'].get('display_name', '')    if 'Histopathology' in old_kernel:        nb['metadata']['kernelspec'] = {            "display_name": "Python 3",            "language": "python",            "name": "python3"        }        print(f"  ✓ Fixed kernel metadata")print()# ============================================================================# STEP 5: Save Cleaned Notebook# ============================================================================print("STEP 5: Saving Cleaned Notebook")print("-"*70)clean_path = nb_path.parent / "SafePick_Analysis_Pipeline.ipynb"with open(clean_path, 'w', encoding='utf-8') as f:    json.dump(nb, f, indent=1, ensure_ascii=False)print(f"✅ Cleaned notebook: {clean_path}\n")# ============================================================================# STEP 6: Create Supporting Files# ============================================================================print("STEP 6: Creating Supporting Files")print("-"*70)output_dir = nb_path.parent / "SafePick_GitHub_Upload"output_dir.mkdir(exist_ok=True)# 6.1: Copy cleaned notebookshutil.copy(clean_path, output_dir / "SafePick_Analysis_Pipeline.ipynb")print(f"  ✓ Copied notebook to: {output_dir.name}/")# 6.2: Create READMEcurrent_date = datetime.now().strftime('%Y-%m-%d')readme = f'''# SafePick: Guard-Constrained Robot Escort Policy EvaluationAnalysis framework for warehouse robot escort policies under safety constraints.## Quick Start```bashgit clone https://github.com/Sjtu-Fuxilab/Guard-Constrained-HRC.gitcd Guard-Constrained-HRCpip install -r requirements.txtexport SAFEPICK_DATA="/path/to/data"jupyter notebook SafePick_Analysis_Pipeline.ipynb```## Key Results- Ergonomic improvement: 5.4-5.8% ΔEHB reduction- Median time impact: ≤0.34%- Adoption rate (ε=1.0%): 62.2%- Digital twin MAPE: 2.96%## DataDownload from: https://doi.org/10.17632/pf2w725pw3.1## Citation```bibtex@article{{safepick2025,  title={{Guard-Constrained Robot Escort Policies for Warehouse Intralogistics}},  author={{Zafar, Sanwal Ahmad and Qin, Wei and Chengliang, Liu and Chen, Tiger and Tan, Runzhi}},  year={{2025}}}}```## Contact- PI: Wei Qin (wqin@sjtu.edu.cn)- Institution: Shanghai Jiao Tong UniversityGenerated: {current_date}'''(output_dir / "README.md").write_text(readme, encoding='utf-8')print(f"  ✓ Created: README.md")# 6.3: Create requirements.txtrequirements = '''numpy>=1.24.0pandas>=2.0.0scipy>=1.10.0matplotlib>=3.7.0seaborn>=0.12.0scikit-learn>=1.3.0pyarrow>=12.0.0jupyter>=1.0.0tqdm>=4.65.0'''(output_dir / "requirements.txt").write_text(requirements, encoding='utf-8')print(f"  ✓ Created: requirements.txt")# 6.4: Create .gitignoregitignore = '''# Data filesdata/results/*.parquet*.csv*.xlsx# Python__pycache__/*.py[cod].ipynb_checkpoints/*.pyc# Environmentsvenv/.env# OS.DS_StoreThumbs.db# Logs*.log# Backups*_backup_*.ipynb'''(output_dir / ".gitignore").write_text(gitignore, encoding='utf-8')print(f"  ✓ Created: .gitignore")# 6.5: Create data directory(output_dir / "data").mkdir(exist_ok=True)(output_dir / "data" / "README.md").write_text(    "# Data Directory\n\nDownload dataset from: https://doi.org/10.17632/pf2w725pw3.1\n",    encoding='utf-8')print(f"  ✓ Created: data/README.md")print()# ============================================================================# STEP 7: Create Upload Instructions# ============================================================================print("STEP 7: Creating Upload Instructions")print("-"*70)instructions = f'''# How to Upload to GitHub## Files Ready for UploadAll files are in: {output_dir.name}/- SafePick_Analysis_Pipeline.ipynb (cleaned notebook)- README.md- requirements.txt- .gitignore- data/README.md## Upload Methods### Method 1: Using Git (from your local machine)```bash# 1. Download the folder '{output_dir.name}' to your local machine# 2. Open terminal in that folder# 3. Run these commands:git initgit add .git commit -m "Initial commit: SafePick analysis pipeline"git remote add origin https://github.com/Sjtu-Fuxilab/Guard-Constrained-HRC.gitgit branch -M maingit push -u origin main```If repository already has content:```bashgit push -u origin main --force```### Method 2: Using GitHub Web Interface1. Go to: https://github.com/Sjtu-Fuxilab/Guard-Constrained-HRC2. Click: "Add file" → "Upload files"3. Drag all files from '{output_dir.name}' folder4. Commit message: "Initial commit: SafePick analysis pipeline"5. Click: "Commit changes"### Method 3: Using GitHub Desktop1. Open GitHub Desktop2. File → New Repository3. Choose the '{output_dir.name}' folder4. Publish to GitHub5. Choose: Sjtu-Fuxilab/Guard-Constrained-HRC## What Was Cleaned✅ Removed Cell 18 (PANDA code): {stats['removed_panda']}✅ Cleared outputs: {stats['cleared_outputs']} cells✅ Fixed hardcoded paths: {stats['fixed_paths']} cells✅ Fixed informal language: {stats['fixed_informal']} cells✅ Fixed deprecation warnings: {stats['fixed_deprecations']} cells## Next Steps1. Download '{output_dir.name}' folder to your local machine2. Use one of the upload methods above3. Verify files on GitHub: https://github.com/Sjtu-Fuxilab/Guard-Constrained-HRC4. Done! ✅'''(output_dir / "UPLOAD_INSTRUCTIONS.txt").write_text(instructions, encoding='utf-8')print(f"  ✓ Created: UPLOAD_INSTRUCTIONS.txt")print()# ============================================================================# FINAL SUMMARY# ============================================================================print("="*70)print(" CLEANUP COMPLETE ".center(70, "="))print("="*70)print()print(f"✅ Original notebook backed up: {bak.name}")print(f"✅ Cleaned notebook created: {clean_path.name}")print(f"✅ All files ready in: {output_dir}/")print()print("Changes applied:")print(f"  • Removed Cell 18 (PANDA): {'Yes' if stats['removed_panda'] else 'No'}")print(f"  • Cleared outputs: {stats['cleared_outputs']} cells")print(f"  • Fixed hardcoded paths: {stats['fixed_paths']} cells → BASE")print(f"  • Fixed informal language: {stats['fixed_informal']} cells")print(f"  • Fixed deprecation warnings: {stats['fixed_deprecations']} cells")print()print("="*70)print(" NEXT STEPS ".center(70, "="))print("="*70)print()print(f"1. Download the folder: {output_dir}/")print(f"   (All cleaned files are inside)")print()print(f"2. On your LOCAL MACHINE (where GitHub works):")print(f"   Open terminal in the downloaded folder and run:")print()print(f"   cd {output_dir.name}")print(f"   git init")print(f"   git add .")print(f'   git commit -m "Initial commit: SafePick pipeline"')print(f"   git remote add origin https://github.com/Sjtu-Fuxilab/Guard-Constrained-HRC.git")print(f"   git branch -M main")print(f"   git push -u origin main")print()print(f"   OR use GitHub web interface to upload files")print()print(f"3. Read: {output_dir}/UPLOAD_INSTRUCTIONS.txt")print(f"   (Contains detailed upload methods)")print()print("="*70)print()print("🎉 All cleaned and ready! Just download and upload from your local machine.")print()